In [1]:
import numpy as np
import pandas as pd
import math
from math import *

# Création scénarios

In [2]:
def demande_énergétique(année_départ, année_fin, traffic, growth, mj_rpk, techno, nom):
    E=[]
    FK=[]
    PtL=[]
    Bio=[]
    LH2=[]
    E.append(traffic*mj_rpk)
    i_max=année_fin-année_départ
    i=0
    while i<i_max:
            traffic=traffic*(1+growth)
            mj_rpk=mj_rpk*(1-techno)
            E.append(traffic*mj_rpk)
            i=i+1
    if nom=='100FK':
        i=0
        while i<28:
            FK.append(E[i])
            PtL=[0]*28
            Bio=[0]*28
            LH2=[0]*28
            i=i+1
    if nom=='0FK_Bio':
        i=0
        while i<28:
            sig=mono_ramp_up()
            Bio.append(sig[i]*E[i])
            FK.append((1-sig[i])*E[i])
            i=i+1
        PtL=[0]*28
        LH2=[0]*28
    if nom=='0FK_PtL':
        i=0
        while i<28:
            sig=mono_ramp_up()
            PtL.append(sig[i]*E[i])
            FK.append((1-sig[i])*E[i])
            i=i+1
        Bio=[0]*28
        LH2=[0]*28
    if nom=='50FK_Bio':
        i=0
        PtL=[0]*28
        LH2=[0]*28
        while i<28:
            sig=mono_ramp_up()
            Bio.append(sig[i]*E[i]*0.5)
            FK.append(E[i]-Bio[i])
            i=i+1
    if nom=='50FK_PtL':
        i=0
        Bio=[0]*28
        LH2=[0]*28
        while i<28:
            sig=mono_ramp_up()
            PtL.append(sig[i]*E[i]*0.5)
            FK.append(E[i]-PtL[i])
            i=i+1
    if nom=='Refuel_EU type':
        i=0
        LH2=[0]*28
        while i<28:
            FK.append(Refuel_EU_ramp_up("Non")[0][i]*E[i])
            PtL.append(Refuel_EU_ramp_up("Non")[1][i]*E[i])
            Bio.append(Refuel_EU_ramp_up("Non")[2][i]*E[i])
            i=i+1
    if nom=='0FK_PtL&Bio':
        i=0
        LH2=[0]*28
        while i<28:
            FK.append(E[i]*PtL_Bio_ramp_up()[0][i])
            PtL.append(E[i]*PtL_Bio_ramp_up()[1][i])
            Bio.append(E[i]*PtL_Bio_ramp_up()[2][i])
            i=i+1
    if nom=='50FK_PtL&Bio':
        i=0
        LH2=[0]*28
        while i<28:
            PtL.append(E[i]*PtL_Bio_ramp_up()[1][i]*0.5)
            Bio.append(E[i]*PtL_Bio_ramp_up()[2][i]*0.5)
            FK.append(E[i]-(PtL[i]+Bio[i]))
            i=i+1
    if nom=='0FK_Full':
        i=0
        while i<28:
            PtL.append(E[i]*Full_ramp_up()[1][i])
            Bio.append(E[i]*Full_ramp_up()[2][i])
            LH2.append(E[i]*Refuel_EU_ramp_up("Oui")[3][i])
            FK.append(E[i]-(PtL[i]+Bio[i]+LH2[i]))
            i=i+1
    if nom=='50FK_Full':
        i=0
        while i<28:
            PtL.append(E[i]*Full_ramp_up()[1][i]*0.5)
            Bio.append(E[i]*Full_ramp_up()[2][i]*0.5)
            LH2.append(E[i]*Refuel_EU_ramp_up("Oui")[3][i]*0.5)
            FK.append(E[i]-(PtL[i]+Bio[i]+LH2[i]))
            i=i+1
    return FK, PtL, Bio, LH2
    
def Refuel_EU_ramp_up(lh2):
    year=[i+2023 for i in range(28)]
    refuel_x=(2023,2025,2030,2032,2035,2040,2045,2050)
    ptl_y=(0,0,0.007,0.012,0.05,0.10,0.15,0.35)
    bio_y=(0,0.02,0.053,0.048,0.15,0.24,0.27,0.35)
    lh2_x=(2035,2050)
    lh2_y=(0,0.05)
    ptl_ramp_up=(np.interp(year,refuel_x,ptl_y)).tolist()
    bio_ramp_up=(np.interp(year,refuel_x,bio_y)).tolist()
    lh2_ramp_up=(np.interp(year,lh2_x,lh2_y)).tolist()
    fk_ramp_up=[]
    i=0
    if lh2=="Non":
        while i<28:
            fk_ramp_up.append(1-(ptl_ramp_up[i]+bio_ramp_up[i]))
            i=i+1
    if lh2=="Oui":
        while i<28:
            fk_ramp_up.append(1-(ptl_ramp_up[i]+bio_ramp_up[i]+lh2_ramp_up[i]))
            i=i+1
    return fk_ramp_up, ptl_ramp_up, bio_ramp_up, lh2_ramp_up
            
def mono_ramp_up():
    interp=[i for i in range(28)]
    xp=(0,28)
    yp=(-5,5)
    x = np.interp(interp,xp,yp)
    z = np.exp(-x)
    sig = (1 / (1 + z)).tolist()
    return sig
    
def PtL_Bio_ramp_up():
    PtL_final=0.73
    Bio_final=0.27
    ptl_ramp_up=[]
    bio_ramp_up=[]
    fk_ramp_up=[]
    i=0
    while i<28:
        ptl_ramp_up.append(PtL_final*(Refuel_EU_ramp_up("Non")[1][i]/Refuel_EU_ramp_up("Non")[1][27]))
        bio_ramp_up.append(Bio_final*(Refuel_EU_ramp_up("Non")[2][i]/Refuel_EU_ramp_up("Non")[2][27]))
        fk_ramp_up.append(1-(ptl_ramp_up[i]+bio_ramp_up[i]))   
        i=i+1
    return fk_ramp_up, ptl_ramp_up, bio_ramp_up

def Full_ramp_up():
    PtL_final=0.2
    Bio_final=0.75
    ptl_ramp_up=[]
    bio_ramp_up=[]
    fk_ramp_up=[]
    i=0
    while i<28:
        ptl_ramp_up.append(PtL_final*(Refuel_EU_ramp_up("Oui")[1][i]/Refuel_EU_ramp_up("Oui")[1][27]))
        bio_ramp_up.append(Bio_final*(Refuel_EU_ramp_up("Oui")[2][i]/Refuel_EU_ramp_up("Oui")[2][27]))
        fk_ramp_up.append(1-(ptl_ramp_up[i]+bio_ramp_up[i]))    
        i=i+1
    return fk_ramp_up, ptl_ramp_up, bio_ramp_up

# Import résultats LCA par MJ

In [3]:
df=pd.read_excel(r"D:\b.pais\Documents\Articles\Resultats\Data_automatisation.xlsx")

# Sélections AEC

In [4]:
def selection_AEC(PtL, Bio):
    if PtL=="wind":
        N_PtL=df["N_PtL_wind"].tolist()
        P_PtL=df["P_PtL_wind"].tolist()
        LSC_PtL=df["LSC_PtL_wind"].tolist()
        O3_PtL=df["O3_PtL_wind"].tolist()
        BII_DLU_PtL=df["BII_DLU_PtL_wind"].tolist()
        FWU_PtL=df["FWU_PtL_wind"].tolist()
        CO2_PtL=df["CO2_PtL_wind"].tolist()
        CO2_non_fossil_PtL=df["CO2_non_fossil_PtL_wind"].tolist()
        CO2_capture_PtL=df["CO2_capture_PtL_wind"].tolist()
        CH4_PtL=df["CH4_PtL_wind"].tolist()
        N2O_PtL=df["N2O_PtL_wind"].tolist()
    if PtL=="PV":
        N_PtL=df["N_PtL_PV"].tolist()
        P_PtL=df["P_PtL_PV"].tolist()
        LSC_PtL=df["LSC_PtL_PV"].tolist()
        O3_PtL=df["O3_PtL_PV"].tolist()
        BII_DLU_PtL=df["BII_DLU_PtL_PV"].tolist()
        FWU_PtL=df["FWU_PtL_PV"].tolist()
        CO2_PtL=df["CO2_PtL_PV"].tolist()
        CO2_non_fossil_PtL=df["CO2_non_fossil_PtL_PV"].tolist()
        CO2_capture_PtL=df["CO2_capture_PtL_PV"].tolist()
        CH4_PtL=df["CH4_PtL_PV"].tolist()
        N2O_PtL=df["N2O_PtL_PV"].tolist()
    if PtL=="WG":
        N_PtL=df["N_PtL_WG"].tolist()
        P_PtL=df["P_PtL_WG"].tolist()
        LSC_PtL=df["LSC_PtL_WG"].tolist()
        O3_PtL=df["O3_PtL_WG"].tolist()
        BII_DLU_PtL=df["BII_DLU_PtL_WG"].tolist()
        FWU_PtL=df["FWU_PtL_WG"].tolist()
        CO2_PtL=df["CO2_PtL_WG"].tolist()
        CO2_non_fossil_PtL=df["CO2_non_fossil_PtL_WG"].tolist()
        CO2_capture_PtL=df["CO2_capture_PtL_WG"].tolist()
        CH4_PtL=df["CH4_PtL_WG"].tolist()
        N2O_PtL=df["N2O_PtL_WG"].tolist()
    if PtL=="Mix":
        i=0
        N_PtL=[]
        P_PtL=[]
        LSC_PtL=[]
        O3_PtL=[]
        BII_DLU_PtL=[]
        FWU_PtL=[]
        CO2_PtL=[]
        CO2_non_fossil_PtL=[]
        CO2_capture_PtL=[]
        CH4_PtL=[]
        N2O_PtL=[]
        while i<4:
            N_PtL.append((df["N_PtL_wind"][i]+df["N_PtL_PV"][i]+df["N_PtL_WG"][i])/3)
            P_PtL.append((df["P_PtL_wind"][i]+df["P_PtL_PV"][i]+df["P_PtL_WG"][i])/3)
            LSC_PtL.append((df["LSC_PtL_wind"][i]+df["LSC_PtL_PV"][i]+df["LSC_PtL_WG"][i])/3)
            O3_PtL.append((df["O3_PtL_wind"][i]+df["O3_PtL_PV"][i]+df["O3_PtL_WG"][i])/3)
            BII_DLU_PtL.append((df["BII_DLU_PtL_wind"][i]+df["BII_DLU_PtL_PV"][i]+df["BII_DLU_PtL_WG"][i])/3)
            FWU_PtL.append((df["FWU_PtL_wind"][i]+df["FWU_PtL_PV"][i]+df["FWU_PtL_WG"][i])/3)
            CO2_PtL.append((df["CO2_PtL_wind"][i]+df["CO2_PtL_PV"][i]+df["CO2_PtL_WG"][i])/3)
            CO2_non_fossil_PtL.append((df["CO2_non_fossil_PtL_wind"][i]+df["CO2_non_fossil_PtL_PV"][i]+df["CO2_non_fossil_PtL_WG"][i])/3)
            CO2_capture_PtL.append((df["CO2_capture_PtL_wind"][i]+df["CO2_capture_PtL_PV"][i]+df["CO2_capture_PtL_WG"][i])/3)
            CH4_PtL.append((df["CH4_PtL_wind"][i]+df["CH4_PtL_PV"][i]+df["CH4_PtL_WG"][i])/3)
            N2O_PtL.append((df["N2O_PtL_wind"][i]+df["N2O_PtL_PV"][i]+df["N2O_PtL_WG"][i])/3)
            i=i+1
    if Bio=="FT residues":
        N_Bio=df["N_Bio_residues"].tolist()
        P_Bio=df["P_Bio_residues"].tolist()
        LSC_Bio=df["LSC_Bio_residues"].tolist()
        O3_Bio=df["O3_Bio_residues"].tolist()
        BII_DLU_Bio=df["BII_DLU_Bio_residues"].tolist()
        FWU_Bio=df["FWU_Bio_residues"].tolist()
        CO2_Bio=df["CO2_Bio_residues"].tolist()
        CO2_non_fossil_Bio=df["CO2_non_fossil_Bio_residues"].tolist()
        CO2_capture_Bio=df["CO2_capture_Bio_residues"].tolist()
        CH4_Bio=df["CH4_Bio_residues"].tolist()
        N2O_Bio=df["N2O_Bio_residues"].tolist()
    if Bio=="FT products":
        N_Bio=df["N_Bio_products"].tolist()
        P_Bio=df["P_Bio_products"].tolist()
        LSC_Bio=df["LSC_Bio_products"].tolist()
        O3_Bio=df["O3_Bio_products"].tolist()
        BII_DLU_Bio=df["BII_DLU_Bio_products"].tolist()
        FWU_Bio=df["FWU_Bio_products"].tolist()
        CO2_Bio=df["CO2_Bio_products"].tolist()
        CO2_non_fossil_Bio=df["CO2_non_fossil_Bio_products"].tolist()
        CO2_capture_Bio=df["CO2_capture_Bio_products"].tolist()
        CH4_Bio=df["CH4_Bio_products"].tolist()
        N2O_Bio=df["N2O_Bio_products"].tolist()
    if Bio=="FT mix":
        i=0
        N_Bio=[]
        P_Bio=[]
        LSC_Bio=[]
        O3_Bio=[]
        BII_DLU_Bio=[]
        FWU_Bio=[]
        CO2_Bio=[]
        CO2_non_fossil_Bio=[]
        CO2_capture_Bio=[]
        CH4_Bio=[]
        N2O_Bio=[]
        while i<4:
            N_Bio.append((df["N_Bio_residues"][i]+df["N_Bio_products"][i])/2)
            P_Bio.append((df["P_Bio_residues"][i]+df["P_Bio_products"][i])/2)
            LSC_Bio.append((df["LSC_Bio_residues"][i]+df["LSC_Bio_products"][i])/2)
            O3_Bio.append((df["O3_Bio_residues"][i]+df["O3_Bio_products"][i])/2)
            BII_DLU_Bio.append((df["BII_DLU_Bio_residues"][i]+df["BII_DLU_Bio_products"][i])/2)
            FWU_Bio.append((df["FWU_Bio_residues"][i]+df["FWU_Bio_products"][i])/2)
            CO2_Bio.append((df["CO2_Bio_residues"][i]+df["CO2_Bio_products"][i])/2)
            CO2_non_fossil_Bio.append((df["CO2_non_fossil_Bio_residues"][i]+df["CO2_non_fossil_Bio_products"][i])/2)
            CO2_capture_Bio.append((df["CO2_capture_Bio_residues"][i]+df["CO2_capture_Bio_products"][i])/2)
            CH4_Bio.append((df["CH4_Bio_residues"][i]+df["CH4_Bio_products"][i])/2)
            N2O_Bio.append((df["N2O_Bio_residues"][i]+df["N2O_Bio_products"][i])/2)
            i=i+1        
    if Bio=="HEFA UCO PV":
        N_Bio=df["N_Bio_HEFA_UCO_PV"].tolist()
        P_Bio=df["P_Bio_HEFA_UCO_PV"].tolist()
        LSC_Bio=df["LSC_Bio_HEFA_UCO_PV"].tolist()
        O3_Bio=df["O3_Bio_HEFA_UCO_PV"].tolist()
        BII_DLU_Bio=df["BII_DLU_Bio_HEFA_UCO_PV"].tolist()
        FWU_Bio=df["FWU_Bio_HEFA_UCO_PV"].tolist()
        CO2_Bio=df["CO2_Bio_HEFA_UCO_PV"].tolist()
        CO2_non_fossil_Bio=df["CO2_non_fossil_Bio_HEFA_UCO_PV"].tolist()
        CO2_capture_Bio=df["CO2_capture_Bio_HEFA_UCO_PV"].tolist()
        CH4_Bio=df["CH4_Bio_HEFA_UCO_PV"].tolist()
        N2O_Bio=df["N2O_Bio_HEFA_UCO_PV"].tolist()
    if Bio=="HEFA UCO wind":
        N_Bio=df["N_Bio_HEFA_UCO_wind"].tolist()
        P_Bio=df["P_Bio_HEFA_UCO_wind"].tolist()
        LSC_Bio=df["LSC_Bio_HEFA_UCO_wind"].tolist()
        O3_Bio=df["O3_Bio_HEFA_UCO_wind"].tolist()
        BII_DLU_Bio=df["BII_DLU_Bio_HEFA_UCO_wind"].tolist()
        FWU_Bio=df["FWU_Bio_HEFA_UCO_wind"].tolist()
        CO2_Bio=df["CO2_Bio_HEFA_UCO_wind"].tolist()
        CO2_non_fossil_Bio=df["CO2_non_fossil_Bio_HEFA_UCO_wind"].tolist()
        CO2_capture_Bio=df["CO2_capture_Bio_HEFA_UCO_wind"].tolist()
        CH4_Bio=df["CH4_Bio_HEFA_UCO_wind"].tolist()
        N2O_Bio=df["N2O_Bio_HEFA_UCO_wind"].tolist()
    if Bio=="HEFA UCO SMR":
        N_Bio=df["N_Bio_HEFA_UCO_SMR"].tolist()
        P_Bio=df["P_Bio_HEFA_UCO_SMR"].tolist()
        LSC_Bio=df["LSC_Bio_HEFA_UCO_SMR"].tolist()
        O3_Bio=df["O3_Bio_HEFA_UCO_SMR"].tolist()
        BII_DLU_Bio=df["BII_DLU_Bio_HEFA_UCO_SMR"].tolist()
        FWU_Bio=df["FWU_Bio_HEFA_UCO_SMR"].tolist()
        CO2_Bio=df["CO2_Bio_HEFA_UCO_SMR"].tolist()
        CO2_non_fossil_Bio=df["CO2_non_fossil_Bio_HEFA_UCO_SMR"].tolist()
        CO2_capture_Bio=df["CO2_capture_Bio_HEFA_UCO_SMR"].tolist()
        CH4_Bio=df["CH4_Bio_HEFA_UCO_SMR"].tolist()
        N2O_Bio=df["N2O_Bio_HEFA_UCO_SMR"].tolist()
    if Bio=="HEFA UCO mix":
        i=0
        N_Bio=[]
        P_Bio=[]
        LSC_Bio=[]
        O3_Bio=[]
        BII_DLU_Bio=[]
        FWU_Bio=[]
        CO2_Bio=[]
        CO2_non_fossil_Bio=[]
        CO2_capture_Bio=[]
        CH4_Bio=[]
        N2O_Bio=[]
        while i<4:
            N_Bio.append((df["N_Bio_HEFA_UCO_wind"][i]+df["N_Bio_HEFA_UCO_PV"][i]+df["N_Bio_HEFA_UCO_SMR"][i])/3)
            P_Bio.append((df["P_Bio_HEFA_UCO_wind"][i]+df["P_Bio_HEFA_UCO_PV"][i]+df["P_Bio_HEFA_UCO_SMR"][i])/3)
            LSC_Bio.append((df["LSC_Bio_HEFA_UCO_wind"][i]+df["LSC_Bio_HEFA_UCO_PV"][i]+df["LSC_Bio_HEFA_UCO_SMR"][i])/3)
            O3_Bio.append((df["O3_Bio_HEFA_UCO_wind"][i]+df["O3_Bio_HEFA_UCO_PV"][i]+df["O3_Bio_HEFA_UCO_SMR"][i])/3)
            BII_DLU_Bio.append((df["BII_DLU_Bio_HEFA_UCO_wind"][i]+df["BII_DLU_Bio_HEFA_UCO_PV"][i]+df["BII_DLU_Bio_HEFA_UCO_SMR"][i])/3)
            FWU_Bio.append((df["FWU_Bio_HEFA_UCO_wind"][i]+df["FWU_Bio_HEFA_UCO_PV"][i]+df["FWU_Bio_HEFA_UCO_SMR"][i])/3)
            CO2_Bio.append((df["CO2_Bio_HEFA_UCO_wind"][i]+df["CO2_Bio_HEFA_UCO_PV"][i]+df["CO2_Bio_HEFA_UCO_SMR"][i])/3)
            CO2_non_fossil_Bio.append((df["CO2_non_fossil_Bio_HEFA_UCO_wind"][i]+df["CO2_non_fossil_Bio_HEFA_UCO_PV"][i]+df["CO2_non_fossil_Bio_HEFA_UCO_SMR"][i])/3)
            CO2_capture_Bio.append((df["CO2_capture_Bio_HEFA_UCO_wind"][i]+df["CO2_capture_Bio_HEFA_UCO_PV"][i]+df["CO2_capture_Bio_HEFA_UCO_SMR"][i])/3)
            CH4_Bio.append((df["CH4_Bio_HEFA_UCO_wind"][i]+df["CH4_Bio_HEFA_UCO_PV"][i]+df["CH4_Bio_HEFA_UCO_SMR"][i])/3)
            N2O_Bio.append((df["N2O_Bio_HEFA_UCO_wind"][i]+df["N2O_Bio_HEFA_UCO_PV"][i]+df["N2O_Bio_HEFA_UCO_SMR"][i])/3)
            i=i+1
    if Bio=="HEFA Rapeseed WG":
        N_Bio=df["N_Bio_HEFA_RS_WG"].tolist()
        P_Bio=df["P_Bio_HEFA_RS_WG"].tolist()
        LSC_Bio=df["LSC_Bio_HEFA_RS_WG"].tolist()
        O3_Bio=df["O3_Bio_HEFA_RS_WG"].tolist()
        BII_DLU_Bio=df["BII_DLU_Bio_HEFA_RS_WG"].tolist()
        FWU_Bio=df["FWU_Bio_HEFA_RS_WG"].tolist()
        CO2_Bio=df["CO2_Bio_HEFA_RS_WG"].tolist()
        CO2_non_fossil_Bio=df["CO2_non_fossil_Bio_HEFA_RS_WG"].tolist()
        CO2_capture_Bio=df["CO2_capture_Bio_HEFA_RS_WG"].tolist()
        CH4_Bio=df["CH4_Bio_HEFA_RS_WG"].tolist()
        N2O_Bio=df["N2O_Bio_HEFA_RS_WG"].tolist()
    
    return N_PtL, P_PtL, LSC_PtL, O3_PtL, BII_DLU_PtL, FWU_PtL, N_Bio, P_Bio, LSC_Bio, O3_Bio, BII_DLU_Bio, FWU_Bio,CO2_PtL, CO2_non_fossil_PtL, CO2_capture_PtL, CH4_PtL, N2O_PtL, CO2_Bio, CO2_non_fossil_Bio, CO2_capture_Bio, CH4_Bio, N2O_Bio
N_LH2=df["N_LH2"].tolist()
P_LH2=df["P_LH2"].tolist()
LSC_LH2=df["LSC_LH2"].tolist()
O3_LH2=df["O3_LH2"].tolist()
BII_DLU_LH2=df["BII_DLU_LH2"].tolist()
FWU_LH2=df["FWU_LH2"].tolist()
CO2_LH2=df["CO2_LH2"].tolist()
CO2_non_fossil_LH2=df["CO2_non_fossil_LH2"].tolist()
CO2_capture_LH2=df["CO2_capture_LH2"].tolist()
CH4_LH2=df["CH4_LH2"].tolist()
N2O_LH2=df["N2O_LH2"].tolist()

N_FK=df["N_FK"].tolist()
P_FK=df["P_FK"].tolist()
LSC_FK=df["LSC_FK"].tolist()
O3_FK=df["O3_FK"].tolist()
BII_DLU_FK=df["BII_DLU_FK"].tolist()
FWU_FK=df["FWU_FK"].tolist()
CO2_FK=df["CO2_FK"].tolist()
CO2_non_fossil_FK=df["CO2_non_fossil_FK"].tolist()
CO2_capture_FK=df["CO2_capture_FK"].tolist()
CH4_FK=df["CH4_FK"].tolist()
N2O_FK=df["N2O_FK"].tolist()

In [5]:
def historical_fossil_kerosene():
    cumulative_CO2_emissions=3.26E10 #tCO2 entre 1940 et 2018 / Source: Lee et al. 2021
    cumulative_fuel_use=2.87E2 #billion gallons entre 2019 et 2022 / Source: IATA
    fossil_kerosene_density=8.04e-1 #kg/L / Source: British Petroleum
    liter_per_gallon=3.785
    co2_per_kg_fk=3.16 #kgCO2/kgFK / Source: Fabre et al. 2021
    fossil_kerosene_energy_content=43.15 #MJ/kg
    cumulative_fuel_use_kg=cumulative_CO2_emissions*1000/co2_per_kg_fk+cumulative_fuel_use*fossil_kerosene_density*liter_per_gallon*1e9
    cumulative_fuel_use_MJ=cumulative_fuel_use_kg*fossil_kerosene_energy_content
    return cumulative_fuel_use_MJ

In [6]:
def aesa(SoSOS, PB):
    result = []
    for i in range(len(PB)):
        result.append(PB[i]/SoSOS)
    return result

In [7]:
SoSOS_FHN_CC=0.0088
SoSOS_FHN_FWU=0.0222
SoSOS_FHN_LSC=0.0246
SoSOS_FHN_P=0.000017057109834137200
SoSOS_FHN_N=0.00016201618133512
SoSOS_FHN_O3=0.00398
SoSOS_FHN_BII=0.00012
SoSOS_GDP=0.041
SOS_PPM=72
SOS_RF=1
SOS_FWU=4000
SOS_LSC=25
SOS_P=9.96
SOS_N=62
SOS_O3=15
SOS_BII=10

# Dynamic climate change and BII impact assessment

### Bern carbon cycle 

In [8]:
#CO2
a0=0.212
a1=0.244
a2=0.336
a3=0.207
alpha1=336.4
alpha2=27.89
alpha3=4.055

#CH4
alpha_methane=12.4

#N2O
alpha_N2O=121

#Molar mass
masse_molaire_CO2=44
masse_molaire_CH4=16.04
masse_molaire_N20=44.013
masse_molaire_air=28.97

#Constants
sens_RF_CO2=5.35
alpha_CH4=0.036
alpha_N2O=0.12

#Atmospheric concentrations (2019)
CO2_ppm_2019=410
CH4_ppb_2019=1866 #ppb
N2O_ppb_2019=332 #ppb

In [20]:
#Fonction
def fraction_CO2_t(tableau):
    i=0
    while i<80:
        calcul_fraction_CO2=a0+a1*math.exp(-i/alpha1)+a2*math.exp(-i/alpha2)+a3*math.exp(-i/alpha3)
        tableau.append(calcul_fraction_CO2)
        calcul_fraction_CO2=0
        i=i+1
        
def fraction_GES_t(tableau, alpha_GES):
    i=0
    while i<80:
        calcul_fraction_GES=np.exp(-i/alpha_GES)
        tableau.append(calcul_fraction_GES)
        i=i+1
        
def calcul_masse_GES(tableau, emission, tableau_fraction_GES):
    x=len(emission)
    b=0
    c=0
    while c<x:
        a=c
        b=0
        calculmasse=0
        while a>=0:
            calculmasse=calculmasse+(emission[a])*tableau_fraction_GES[b]
            a=a-1
            b=b+1
        tableau.append(calculmasse)
        c=c+1
        
def concentration_GES_t(masse_GES_t, tableau_concentration_GES_t, masse_molaire_GES):
    a=0
    b=len(masse_GES_t)
    while a<b:
        concentration_GES_t=masse_GES_t[a]*(1000000/5.15e18)/(masse_molaire_GES/masse_molaire_air)
        tableau_concentration_GES_t.append(concentration_GES_t)
        a=a+1
        
# Forçage radiatif à l'année t - CO2
def RF_CO2_t(tableau_concentration_t, tableau_RF_t, GES_ppm_2019, sens_RF_GES):
    a=0
    b=len(tableau_concentration_t)
    while a<b:
        RF_additionnel=sens_RF_GES*np.log(1+tableau_concentration_t[a]/GES_ppm_2019)
        tableau_RF_t.append(RF_additionnel)
        a=a+1
    
# Baisse forçage radiatif capture - CO2
def RF_CO2_t_capture(tableau_concentration_t, tableau_RF_t, GES_ppm_2019, sens_RF_GES):
    a=0
    b=len(tableau_concentration_t)
    while a<b:
        RF_additionnel=sens_RF_GES*np.log(1-tableau_concentration_t[a]/GES_ppm_2019)
        tableau_RF_t.append(-RF_additionnel)
        a=a+1
        
#Fonction f(M,N)
def f(concentration_CH4, concentration_N2O):
    fMN=0.47*np.log(1+2.01e-5*(concentration_CH4*concentration_N2O)**0.75+5.31e-15*concentration_CH4*(concentration_CH4*concentration_N2O)**1.52) 
    return fMN

#Forçage radiatif à l'année t - CH4
def RF_CH4_t(tableau_concentration_t, tableau_RF_t, CH4_ppb_2019, N2O_ppb_2019, alpha_GES):
    a=0
    b=len(tableau_concentration_t)
    while a<b:
        RF_additionnel=alpha_GES*((CH4_ppb_2019+tableau_concentration_t[a])**(1/2)-CH4_ppb_2019**(1/2))-(f(CH4_ppb_2019+tableau_concentration_t[a], N2O_ppb_2019)-f(CH4_ppb_2019,N2O_ppb_2019))
        tableau_RF_t.append(RF_additionnel)
        a=a+1
        
#Forçage radiatif à l'année t - CH4
def RF_N2O_t(tableau_concentration_t, tableau_RF_t, CH4_ppb_2019, N2O_ppb_2019, alpha_GES):
    a=0
    b=len(tableau_concentration_t)
    while a<b:
        RF_additionnel=alpha_GES*((N2O_ppb_2019+tableau_concentration_t[a])**(1/2)-N2O_ppb_2019**(1/2))-(f(CH4_ppb_2019, tableau_concentration_t[a]+N2O_ppb_2019)-f(CH4_ppb_2019,N2O_ppb_2019))
        tableau_RF_t.append(RF_additionnel)
        a=a+1
    
def dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil, fossil_kerosene_CO2_non_fossil, fossil_kerosene_CO2_capture, fossil_kerosene_CH4, fossil_kerosene_N20,
                                            PtL_CO2_fossil, PtL_CO2_non_fossil, PtL_CO2_capture, PtL_CH4, PtL_N20,
                                            Bio_CO2_fossil, Bio_CO2_non_fossil, Bio_CO2_capture, Bio_CH4, Bio_N20,
                                            LH2_CO2_fossil, LH2_CO2_non_fossil, LH2_CO2_capture, LH2_CH4, LH2_N20):
    tableau_fraction_CO2=[]
    tableau_fraction_CH4=[]
    tableau_fraction_N20=[]
    fraction_CO2_t(tableau_fraction_CO2)
    fraction_GES_t(tableau_fraction_CH4, alpha_methane)
    fraction_GES_t(tableau_fraction_N20, alpha_N2O)
    
    #Fossil kerosene
    tableau_masse_CO2_fossil_kerosene_CO2_fossil=[]
    tableau_masse_CO2_fossil_kerosene_CO2_non_fossil=[]
    tableau_masse_CO2_fossil_kerosene_CO2_capture=[]
    tableau_masse_CH4_fossil_kerosene=[]
    tableau_masse_N20_fossil_kerosene=[]
    tableau_concentration_fossil_kerosene_CO2_fossil=[]
    tableau_concentration_fossil_kerosene_CO2_non_fossil=[]
    tableau_concentration_fossil_kerosene_CO2_capture=[]
    tableau_concentration_fossil_kerosene_CH4=[]
    tableau_concentration_fossil_kerosene_N20=[]
    tableau_RF_fossil_kerosene_CO2_fossil=[]
    tableau_RF_fossil_kerosene_CO2_non_fossil=[]
    tableau_RF_fossil_kerosene_CO2_capture=[]
    tableau_RF_fossil_kerosene_CH4=[]
    tableau_RF_fossil_kerosene_N20=[]
    calcul_masse_GES(tableau_masse_CH4_fossil_kerosene, fossil_kerosene_CH4, tableau_fraction_CH4)
    calcul_masse_GES(tableau_masse_CO2_fossil_kerosene_CO2_fossil, fossil_kerosene_CO2_fossil, tableau_fraction_CO2)
    calcul_masse_GES(tableau_masse_CO2_fossil_kerosene_CO2_non_fossil, fossil_kerosene_CO2_non_fossil, tableau_fraction_CO2)
    calcul_masse_GES(tableau_masse_CO2_fossil_kerosene_CO2_capture, fossil_kerosene_CO2_capture, tableau_fraction_CO2)
    calcul_masse_GES(tableau_masse_N20_fossil_kerosene, fossil_kerosene_N20, tableau_fraction_N20)
    concentration_GES_t(tableau_masse_CO2_fossil_kerosene_CO2_fossil, tableau_concentration_fossil_kerosene_CO2_fossil, masse_molaire_CO2)
    concentration_GES_t(tableau_masse_CO2_fossil_kerosene_CO2_non_fossil, tableau_concentration_fossil_kerosene_CO2_non_fossil, masse_molaire_CO2)
    concentration_GES_t(tableau_masse_CO2_fossil_kerosene_CO2_capture, tableau_concentration_fossil_kerosene_CO2_capture, masse_molaire_CO2)
    concentration_GES_t(tableau_masse_CH4_fossil_kerosene, tableau_concentration_fossil_kerosene_CH4, masse_molaire_CH4)
    concentration_GES_t(tableau_masse_N20_fossil_kerosene, tableau_concentration_fossil_kerosene_N20, masse_molaire_N20)
    RF_CO2_t(tableau_concentration_fossil_kerosene_CO2_fossil, tableau_RF_fossil_kerosene_CO2_fossil, CO2_ppm_2019, sens_RF_CO2)
    RF_CO2_t(tableau_concentration_fossil_kerosene_CO2_non_fossil, tableau_RF_fossil_kerosene_CO2_non_fossil, CO2_ppm_2019, sens_RF_CO2)
    RF_CO2_t_capture(tableau_concentration_fossil_kerosene_CO2_capture, tableau_RF_fossil_kerosene_CO2_capture, CO2_ppm_2019, sens_RF_CO2)
    RF_CH4_t(tableau_concentration_fossil_kerosene_CH4, tableau_RF_fossil_kerosene_CH4, CH4_ppb_2019, N2O_ppb_2019, alpha_CH4)
    RF_N2O_t(tableau_concentration_fossil_kerosene_N20, tableau_RF_fossil_kerosene_N20, CH4_ppb_2019, N2O_ppb_2019, alpha_N2O)
    #Electro-kerosene
    tableau_masse_CO2_PtL_CO2_fossil=[]
    tableau_masse_CO2_PtL_CO2_non_fossil=[]
    tableau_masse_CO2_PtL_CO2_capture=[]
    tableau_masse_CH4_PtL=[]
    tableau_masse_N20_PtL=[]
    tableau_concentration_PtL_CO2_fossil=[]
    tableau_concentration_PtL_CO2_non_fossil=[]
    tableau_concentration_PtL_CO2_capture=[]
    tableau_concentration_PtL_CH4=[]
    tableau_concentration_PtL_N20=[]
    tableau_RF_PtL_CO2_fossil=[]
    tableau_RF_PtL_CO2_non_fossil=[]
    tableau_RF_PtL_CO2_capture=[]
    tableau_RF_PtL_CH4=[]
    tableau_RF_PtL_N20=[]
    calcul_masse_GES(tableau_masse_CH4_PtL, PtL_CH4, tableau_fraction_CH4)
    calcul_masse_GES(tableau_masse_CO2_PtL_CO2_fossil, PtL_CO2_fossil, tableau_fraction_CO2)
    calcul_masse_GES(tableau_masse_CO2_PtL_CO2_non_fossil, PtL_CO2_non_fossil, tableau_fraction_CO2)
    calcul_masse_GES(tableau_masse_CO2_PtL_CO2_capture, PtL_CO2_capture, tableau_fraction_CO2)
    calcul_masse_GES(tableau_masse_N20_PtL, PtL_N20, tableau_fraction_N20)
    concentration_GES_t(tableau_masse_CO2_PtL_CO2_fossil, tableau_concentration_PtL_CO2_fossil, masse_molaire_CO2)
    concentration_GES_t(tableau_masse_CO2_PtL_CO2_non_fossil, tableau_concentration_PtL_CO2_non_fossil, masse_molaire_CO2)
    concentration_GES_t(tableau_masse_CO2_PtL_CO2_capture, tableau_concentration_PtL_CO2_capture, masse_molaire_CO2)
    concentration_GES_t(tableau_masse_CH4_PtL, tableau_concentration_PtL_CH4, masse_molaire_CH4)
    concentration_GES_t(tableau_masse_N20_PtL, tableau_concentration_PtL_N20, masse_molaire_N20)
    RF_CO2_t(tableau_concentration_PtL_CO2_fossil, tableau_RF_PtL_CO2_fossil, CO2_ppm_2019, sens_RF_CO2)
    RF_CO2_t(tableau_concentration_PtL_CO2_non_fossil, tableau_RF_PtL_CO2_non_fossil, CO2_ppm_2019, sens_RF_CO2)
    RF_CO2_t_capture(tableau_concentration_PtL_CO2_capture, tableau_RF_PtL_CO2_capture, CO2_ppm_2019, sens_RF_CO2)
    RF_CH4_t(tableau_concentration_PtL_CH4, tableau_RF_PtL_CH4, CH4_ppb_2019, N2O_ppb_2019, alpha_CH4)
    RF_N2O_t(tableau_concentration_PtL_N20, tableau_RF_PtL_N20, CH4_ppb_2019, N2O_ppb_2019, alpha_N2O)
    #Bio-kerosene
    tableau_masse_CO2_Bio_CO2_fossil=[]
    tableau_masse_CO2_Bio_CO2_non_fossil=[]
    tableau_masse_CO2_Bio_CO2_capture=[]
    tableau_masse_CH4_Bio=[]
    tableau_masse_N20_Bio=[]
    tableau_concentration_Bio_CO2_fossil=[]
    tableau_concentration_Bio_CO2_non_fossil=[]
    tableau_concentration_Bio_CO2_capture=[]
    tableau_concentration_Bio_CH4=[]
    tableau_concentration_Bio_N20=[]
    tableau_RF_Bio_CO2_fossil=[]
    tableau_RF_Bio_CO2_non_fossil=[]
    tableau_RF_Bio_CO2_capture=[]
    tableau_RF_Bio_CH4=[]
    tableau_RF_Bio_N20=[]
    calcul_masse_GES(tableau_masse_CH4_Bio, Bio_CH4, tableau_fraction_CH4)
    calcul_masse_GES(tableau_masse_CO2_Bio_CO2_fossil, Bio_CO2_fossil, tableau_fraction_CO2)
    calcul_masse_GES(tableau_masse_CO2_Bio_CO2_non_fossil, Bio_CO2_non_fossil, tableau_fraction_CO2)
    calcul_masse_GES(tableau_masse_CO2_Bio_CO2_capture, Bio_CO2_capture, tableau_fraction_CO2)
    calcul_masse_GES(tableau_masse_N20_Bio, Bio_N20, tableau_fraction_N20)
    concentration_GES_t(tableau_masse_CO2_Bio_CO2_fossil, tableau_concentration_Bio_CO2_fossil, masse_molaire_CO2)
    concentration_GES_t(tableau_masse_CO2_Bio_CO2_non_fossil, tableau_concentration_Bio_CO2_non_fossil, masse_molaire_CO2)
    concentration_GES_t(tableau_masse_CO2_Bio_CO2_capture, tableau_concentration_Bio_CO2_capture, masse_molaire_CO2)
    concentration_GES_t(tableau_masse_CH4_Bio, tableau_concentration_Bio_CH4, masse_molaire_CH4)
    concentration_GES_t(tableau_masse_N20_Bio, tableau_concentration_Bio_N20, masse_molaire_N20)
    RF_CO2_t(tableau_concentration_Bio_CO2_fossil, tableau_RF_Bio_CO2_fossil, CO2_ppm_2019, sens_RF_CO2)
    RF_CO2_t(tableau_concentration_Bio_CO2_non_fossil, tableau_RF_Bio_CO2_non_fossil, CO2_ppm_2019, sens_RF_CO2)
    RF_CO2_t_capture(tableau_concentration_Bio_CO2_capture, tableau_RF_Bio_CO2_capture, CO2_ppm_2019, sens_RF_CO2)
    RF_CH4_t(tableau_concentration_Bio_CH4, tableau_RF_Bio_CH4, CH4_ppb_2019, N2O_ppb_2019, alpha_CH4)
    RF_N2O_t(tableau_concentration_Bio_N20, tableau_RF_Bio_N20, CH4_ppb_2019, N2O_ppb_2019, alpha_N2O)
    #Liquefied H2
    tableau_masse_CO2_LH2_CO2_fossil=[]
    tableau_masse_CO2_LH2_CO2_non_fossil=[]
    tableau_masse_CO2_LH2_CO2_capture=[]
    tableau_masse_CH4_LH2=[]
    tableau_masse_N20_LH2=[]
    tableau_concentration_LH2_CO2_fossil=[]
    tableau_concentration_LH2_CO2_non_fossil=[]
    tableau_concentration_LH2_CO2_capture=[]
    tableau_concentration_LH2_CH4=[]
    tableau_concentration_LH2_N20=[]
    tableau_RF_LH2_CO2_fossil=[]
    tableau_RF_LH2_CO2_non_fossil=[]
    tableau_RF_LH2_CO2_capture=[]
    tableau_RF_LH2_CH4=[]
    tableau_RF_LH2_N20=[]
    calcul_masse_GES(tableau_masse_CH4_LH2, LH2_CH4, tableau_fraction_CH4)
    calcul_masse_GES(tableau_masse_CO2_LH2_CO2_fossil, LH2_CO2_fossil, tableau_fraction_CO2)
    calcul_masse_GES(tableau_masse_CO2_LH2_CO2_non_fossil, LH2_CO2_non_fossil, tableau_fraction_CO2)
    calcul_masse_GES(tableau_masse_CO2_LH2_CO2_capture, LH2_CO2_capture, tableau_fraction_CO2)
    calcul_masse_GES(tableau_masse_N20_LH2, LH2_N20, tableau_fraction_N20)
    concentration_GES_t(tableau_masse_CO2_LH2_CO2_fossil, tableau_concentration_LH2_CO2_fossil, masse_molaire_CO2)
    concentration_GES_t(tableau_masse_CO2_LH2_CO2_non_fossil, tableau_concentration_LH2_CO2_non_fossil, masse_molaire_CO2)
    concentration_GES_t(tableau_masse_CO2_LH2_CO2_capture, tableau_concentration_LH2_CO2_capture, masse_molaire_CO2)
    concentration_GES_t(tableau_masse_CH4_LH2, tableau_concentration_LH2_CH4, masse_molaire_CH4)
    concentration_GES_t(tableau_masse_N20_LH2, tableau_concentration_LH2_N20, masse_molaire_N20)
    RF_CO2_t(tableau_concentration_LH2_CO2_fossil, tableau_RF_LH2_CO2_fossil, CO2_ppm_2019, sens_RF_CO2)
    RF_CO2_t(tableau_concentration_LH2_CO2_non_fossil, tableau_RF_LH2_CO2_non_fossil, CO2_ppm_2019, sens_RF_CO2)
    RF_CO2_t_capture(tableau_concentration_LH2_CO2_capture, tableau_RF_LH2_CO2_capture, CO2_ppm_2019, sens_RF_CO2)
    RF_CH4_t(tableau_concentration_LH2_CH4, tableau_RF_LH2_CH4, CH4_ppb_2019, N2O_ppb_2019, alpha_CH4)
    RF_N2O_t(tableau_concentration_LH2_N20, tableau_RF_LH2_N20, CH4_ppb_2019, N2O_ppb_2019, alpha_N2O)
    
    return  tableau_RF_fossil_kerosene_CO2_fossil,tableau_RF_fossil_kerosene_CO2_non_fossil,tableau_RF_fossil_kerosene_CO2_capture,tableau_RF_fossil_kerosene_CH4,tableau_RF_fossil_kerosene_N20,tableau_RF_PtL_CO2_fossil,tableau_RF_PtL_CO2_non_fossil,tableau_RF_PtL_CO2_capture,tableau_RF_PtL_CH4,tableau_RF_PtL_N20,tableau_RF_Bio_CO2_fossil,tableau_RF_Bio_CO2_non_fossil,tableau_RF_Bio_CO2_capture,tableau_RF_Bio_CH4,tableau_RF_Bio_N20,tableau_RF_LH2_CO2_fossil,tableau_RF_LH2_CO2_non_fossil,tableau_RF_LH2_CO2_capture,tableau_RF_LH2_CH4,tableau_RF_LH2_N20,tableau_concentration_fossil_kerosene_CO2_fossil,tableau_concentration_fossil_kerosene_CO2_non_fossil,tableau_concentration_fossil_kerosene_CO2_capture,tableau_concentration_PtL_CO2_fossil,tableau_concentration_PtL_CO2_non_fossil,tableau_concentration_PtL_CO2_capture,tableau_concentration_Bio_CO2_fossil,tableau_concentration_Bio_CO2_non_fossil,tableau_concentration_Bio_CO2_capture,tableau_concentration_LH2_CO2_fossil,tableau_concentration_LH2_CO2_non_fossil,tableau_concentration_LH2_CO2_capture

In [10]:
MSA_factor=6.31342e12 # % loss m² °C-1 Source: Ha,afiah et al. (2012)
total_area=1.3e14 #total area of the natural and non-natural biomes in the IMAGE model Source: Alkemade et al. (2009)
climate_sens_param=1.06 #climate sensitivity parameter Source: IPCC AR5 and Ryberg et al. (2018)
CF_BII_climate=100*(MSA_factor*climate_sens_param)/total_area

In [11]:
historical_ppm_2018=2.58
historical_ppm_2019_2023=2.95-2.58
historical_rf_2018=34.3/1000
historical_rf_2019_2023=3.92e-2-historical_rf_2018

# Calculs résultats scénarios

In [ ]:
nom_scenario=["100FK", "0FK_PtL", "0FK_Bio", "0FK_PtL&Bio", "0FK_Full", "50FK_PtL", "50FK_Bio", "50FK_PtL&Bio", "50FK_Full", "Refuel_EU type"] #, "0FK_PtL", "0FK_Bio", "0FK_PtL&Bio", "0FK_Full", "50FK_PtL", "50FK_Bio", "50FK_PtL&Bio", "50FK_Full", "Refuel_EU type"
choix_PtL=["wind","PV", "WG", "Mix"]
choix_Bio=["FT residues", "FT products", "FT mix", "HEFA UCO mix", "HEFA UCO wind", "HEFA UCO PV", "HEFA UCO SMR", "HEFA Rapeseed WG"]
growth=float((input("Croissance du trafic:")))
techno=float((input("Amélioration techno:")))
scenario=[]
nature_ptl=[]
nature_bio=[]
n_fk_final=[]
p_fk_final=[]
lsc_fk_final=[]
o3_fk_final=[]
bii_dlu_fk_final=[]
fwu_fk_final=[]
n_ptl_final=[]
p_ptl_final=[]
lsc_ptl_final=[]
o3_ptl_final=[]
bii_dlu_ptl_final=[]
fwu_ptl_final=[]
n_bio_final=[]
p_bio_final=[]
lsc_bio_final=[]
o3_bio_final=[]
bii_dlu_bio_final=[]
fwu_bio_final=[]
n_lh2_final=[]
p_lh2_final=[]
lsc_lh2_final=[]
o3_lh2_final=[]
bii_dlu_lh2_final=[]
fwu_lh2_final=[]
lsc_fk_historical=[]
rf_fk_final=[]
rf_ptl_final=[]
rf_bio_final=[]
rf_lh2_final=[]
ppm_fk_final=[]
ppm_ptl_final=[]
ppm_bio_final=[]
ppm_lh2_final=[]
bii_clim_historical=[]
bii_clim_fk_final=[]
bii_clim_bio_final=[]
bii_clim_ptl_final=[]
bii_clim_lh2_final=[]
rf_historical=[]
ppm_historical=[]
for nom in nom_scenario:
    for ptl in choix_PtL:
        for bio in choix_Bio:

            FK=demande_énergétique(2023,2050,8759364769224,growth,12400000000000/8759364769224,techno,nom)[0]
            PtL=demande_énergétique(2023,2050,8759364769224,growth,12400000000000/8759364769224,techno,nom)[1]
            Bio=demande_énergétique(2023,2050,8759364769224,growth,12400000000000/8759364769224,techno,nom)[2]
            LH2=demande_énergétique(2023,2050,8759364769224,growth,12400000000000/8759364769224,techno,nom)[3]

            N_PtL=selection_AEC(ptl, bio)[0]
            P_PtL=selection_AEC(ptl, bio)[1]
            LSC_PtL=selection_AEC(ptl, bio)[2]
            O3_PtL=selection_AEC(ptl, bio)[3]
            BII_DLU_PtL=selection_AEC(ptl, bio)[4]
            FWU_PtL=selection_AEC(ptl, bio)[5]
            N_Bio=selection_AEC(ptl, bio)[6]
            P_Bio=selection_AEC(ptl, bio)[7]
            LSC_Bio=selection_AEC(ptl, bio)[8]
            O3_Bio=selection_AEC(ptl, bio)[9]
            BII_DLU_Bio=selection_AEC(ptl, bio)[10]
            FWU_Bio=selection_AEC(ptl, bio)[11]
            CO2_PtL=selection_AEC(ptl, bio)[12]
            CO2_non_fossil_PtL=selection_AEC(ptl, bio)[13]
            CO2_capture_PtL=selection_AEC(ptl, bio)[14]
            CH4_PtL=selection_AEC(ptl, bio)[15]
            N2O_PtL=selection_AEC(ptl,bio)[16]
            CO2_Bio=selection_AEC(ptl, bio)[17]
            CO2_non_fossil_Bio=selection_AEC(ptl, bio)[18]
            CO2_capture_Bio=selection_AEC(ptl, bio)[19]
            CH4_Bio=selection_AEC(ptl, bio)[20]
            N2O_Bio=selection_AEC(ptl,bio)[21]

            #N Cycle
            i=0
            N_FK_impact=[]
            N_PtL_impact=[]
            N_Bio_impact=[]
            N_LH2_impact=[]
            while i<7:
                N_FK_impact.append(FK[i]*N_FK[0])
                N_PtL_impact.append(PtL[i]*N_PtL[0])
                N_Bio_impact.append(Bio[i]*N_Bio[0])
                N_LH2_impact.append(LH2[i]*N_LH2[0])
                i=i+1
            while i<17:
                N_FK_impact.append(FK[i]*N_FK[1])
                N_PtL_impact.append(PtL[i]*N_PtL[1])
                N_Bio_impact.append(Bio[i]*N_Bio[1])
                N_LH2_impact.append(LH2[i]*N_LH2[1])
                i=i+1
            while i<27:
                N_FK_impact.append(FK[i]*N_FK[2])
                N_PtL_impact.append(PtL[i]*N_PtL[2])
                N_Bio_impact.append(Bio[i]*N_Bio[2])
                N_LH2_impact.append(LH2[i]*N_LH2[2])
                i=i+1
            while i<28:
                N_FK_impact.append(FK[i]*N_FK[3])
                N_PtL_impact.append(PtL[i]*N_PtL[3])
                N_Bio_impact.append(Bio[i]*N_Bio[3])
                N_LH2_impact.append(LH2[i]*N_LH2[3])
                i=i+1
                
                
            #P Cycle
            i=0
            P_FK_impact=[]
            P_PtL_impact=[]
            P_Bio_impact=[]
            P_LH2_impact=[]
            while i<7:
                P_FK_impact.append(FK[i]*P_FK[0])
                P_PtL_impact.append(PtL[i]*P_PtL[0])
                P_Bio_impact.append(Bio[i]*P_Bio[0])
                P_LH2_impact.append(LH2[i]*P_LH2[0])
                i=i+1
            while i<17:
                P_FK_impact.append(FK[i]*P_FK[1])
                P_PtL_impact.append(PtL[i]*P_PtL[1])
                P_Bio_impact.append(Bio[i]*P_Bio[1])
                P_LH2_impact.append(LH2[i]*P_LH2[1])
                i=i+1
            while i<27:
                P_FK_impact.append(FK[i]*P_FK[2])
                P_PtL_impact.append(PtL[i]*P_PtL[2])
                P_Bio_impact.append(Bio[i]*P_Bio[2])
                P_LH2_impact.append(LH2[i]*P_LH2[2])
                i=i+1
            while i<28:
                P_FK_impact.append(FK[i]*P_FK[3])
                P_PtL_impact.append(PtL[i]*P_PtL[3])
                P_Bio_impact.append(Bio[i]*P_Bio[3])
                P_LH2_impact.append(LH2[i]*P_LH2[3])
                i=i+1
            
                
            #Land system change
            i=0
            LSC_FK_impact=[]
            LSC_PtL_impact=[]
            LSC_Bio_impact=[]
            LSC_LH2_impact=[]
            while i<7:
                LSC_FK_impact.append(FK[i]*LSC_FK[0])
                LSC_PtL_impact.append(PtL[i]*LSC_PtL[0])
                LSC_Bio_impact.append(Bio[i]*LSC_Bio[0])
                LSC_LH2_impact.append(LH2[i]*LSC_LH2[0])
                i=i+1
            while i<17:
                LSC_FK_impact.append(FK[i]*LSC_FK[1])
                LSC_PtL_impact.append(PtL[i]*LSC_PtL[1])
                LSC_Bio_impact.append(Bio[i]*LSC_Bio[1])
                LSC_LH2_impact.append(LH2[i]*LSC_LH2[1])
                i=i+1
            while i<27:
                LSC_FK_impact.append(FK[i]*LSC_FK[2])
                LSC_PtL_impact.append(PtL[i]*LSC_PtL[2])
                LSC_Bio_impact.append(Bio[i]*LSC_Bio[2])
                LSC_LH2_impact.append(LH2[i]*LSC_LH2[2])
                i=i+1
            while i<28:
                LSC_FK_impact.append(FK[i]*LSC_FK[3])
                LSC_PtL_impact.append(PtL[i]*LSC_PtL[3])
                LSC_Bio_impact.append(Bio[i]*LSC_Bio[3])
                LSC_LH2_impact.append(LH2[i]*LSC_LH2[3])
                i=i+1
            LSC_FK_impact_2050=sum(LSC_FK_impact)
            LSC_PtL_impact_2050=sum(LSC_PtL_impact)
            LSC_Bio_impact_2050=sum(LSC_Bio_impact)
            LSC_LH2_impact_2050=sum(LSC_LH2_impact)
            
            #Stratospheric ozone depletion
            i=0
            O3_FK_impact=[]
            O3_PtL_impact=[]
            O3_Bio_impact=[]
            O3_LH2_impact=[]
            while i<7:
                O3_FK_impact.append(FK[i]*O3_FK[0])
                O3_PtL_impact.append(PtL[i]*O3_PtL[0])
                O3_Bio_impact.append(Bio[i]*O3_Bio[0])
                O3_LH2_impact.append(LH2[i]*O3_LH2[0])
                i=i+1
            while i<17:
                O3_FK_impact.append(FK[i]*O3_FK[1])
                O3_PtL_impact.append(PtL[i]*O3_PtL[1])
                O3_Bio_impact.append(Bio[i]*O3_Bio[1])
                O3_LH2_impact.append(LH2[i]*O3_LH2[1])
                i=i+1
            while i<27:
                O3_FK_impact.append(FK[i]*O3_FK[2])
                O3_PtL_impact.append(PtL[i]*O3_PtL[2])
                O3_Bio_impact.append(Bio[i]*O3_Bio[2])
                O3_LH2_impact.append(LH2[i]*O3_LH2[2])
                i=i+1
            while i<28:
                O3_FK_impact.append(FK[i]*O3_FK[3])
                O3_PtL_impact.append(PtL[i]*O3_PtL[3])
                O3_Bio_impact.append(Bio[i]*O3_Bio[3])
                O3_LH2_impact.append(LH2[i]*O3_LH2[3])
                i=i+1
            O3_FK_impact_2050=sum(O3_FK_impact)
            O3_PtL_impact_2050=sum(O3_PtL_impact)
            O3_Bio_impact_2050=sum(O3_Bio_impact)
            O3_LH2_impact_2050=sum(O3_LH2_impact)
            
            #BII Direct land use
            i=0
            BII_DLU_FK_impact=[]
            BII_DLU_PtL_impact=[]
            BII_DLU_Bio_impact=[]
            BII_DLU_LH2_impact=[]
            while i<7:
                BII_DLU_FK_impact.append(FK[i]*BII_DLU_FK[0])
                BII_DLU_PtL_impact.append(PtL[i]*BII_DLU_PtL[0])
                BII_DLU_Bio_impact.append(Bio[i]*BII_DLU_Bio[0])
                BII_DLU_LH2_impact.append(LH2[i]*BII_DLU_LH2[0])
                i=i+1
            while i<17:
                BII_DLU_FK_impact.append(FK[i]*BII_DLU_FK[1])
                BII_DLU_PtL_impact.append(PtL[i]*BII_DLU_PtL[1])
                BII_DLU_Bio_impact.append(Bio[i]*BII_DLU_Bio[1])
                BII_DLU_LH2_impact.append(LH2[i]*BII_DLU_LH2[1])
                i=i+1
            while i<27:
                BII_DLU_FK_impact.append(FK[i]*BII_DLU_FK[2])
                BII_DLU_PtL_impact.append(PtL[i]*BII_DLU_PtL[2])
                BII_DLU_Bio_impact.append(Bio[i]*BII_DLU_Bio[2])
                BII_DLU_LH2_impact.append(LH2[i]*BII_DLU_LH2[2])
                i=i+1
            while i<28:
                BII_DLU_FK_impact.append(FK[i]*BII_DLU_FK[3])
                BII_DLU_PtL_impact.append(PtL[i]*BII_DLU_PtL[3])
                BII_DLU_Bio_impact.append(Bio[i]*BII_DLU_Bio[3])
                BII_DLU_LH2_impact.append(LH2[i]*BII_DLU_LH2[3])
                i=i+1
             
                
            #Freshwater use
            i=0
            FWU_FK_impact=[]
            FWU_PtL_impact=[]
            FWU_Bio_impact=[]
            FWU_LH2_impact=[]
            while i<7:
                FWU_FK_impact.append(FK[i]*FWU_FK[0])
                FWU_PtL_impact.append(PtL[i]*FWU_PtL[0])
                FWU_Bio_impact.append(Bio[i]*FWU_Bio[0])
                FWU_LH2_impact.append(LH2[i]*FWU_LH2[0])
                i=i+1  
            while i<17:
                FWU_FK_impact.append(FK[i]*FWU_FK[1])
                FWU_PtL_impact.append(PtL[i]*FWU_PtL[1])
                FWU_Bio_impact.append(Bio[i]*FWU_Bio[1])
                FWU_LH2_impact.append(LH2[i]*FWU_LH2[1])
                i=i+1 
            while i<27:
                FWU_FK_impact.append(FK[i]*FWU_FK[2])
                FWU_PtL_impact.append(PtL[i]*FWU_PtL[2])
                FWU_Bio_impact.append(Bio[i]*FWU_Bio[2])
                FWU_LH2_impact.append(LH2[i]*FWU_LH2[2])
                i=i+1 
            while i<28:
                FWU_FK_impact.append(FK[i]*FWU_FK[3])
                FWU_PtL_impact.append(PtL[i]*FWU_PtL[3])
                FWU_Bio_impact.append(Bio[i]*FWU_Bio[3])
                FWU_LH2_impact.append(LH2[i]*FWU_LH2[3])
                i=i+1 
                
                
            #Climate change
            fossil_kerosene_CO2_fossil=[]
            fossil_kerosene_CO2_non_fossil=[]
            fossil_kerosene_CO2_capture=[]
            fossil_kerosene_CH4=[]
            fossil_kerosene_N20=[]
            PtL_CO2_fossil=[]
            PtL_CO2_non_fossil=[]
            PtL_CO2_capture=[]
            PtL_CH4=[]
            PtL_N20=[]
            Bio_CO2_fossil=[]
            Bio_CO2_non_fossil=[]
            Bio_CO2_capture=[]
            Bio_CH4=[]
            Bio_N20=[]
            LH2_CO2_fossil=[]
            LH2_CO2_non_fossil=[]
            LH2_CO2_capture=[]
            LH2_CH4=[]
            LH2_N20=[]
            RF_CO2_FK=[]
            RF_CO2_non_fossil_FK=[]
            RF_CO2_capture_FK=[]
            RF_CH4_FK=[]
            RF_N2O_FK=[]
            RF_CO2_PtL=[]
            RF_CO2_non_fossil_PtL=[]
            RF_CO2_capture_PtL=[]
            RF_CH4_PtL=[]
            RF_N2O_PtL=[]
            RF_CO2_Bio=[]
            RF_CO2_non_fossil_Bio=[]
            RF_CO2_capture_Bio=[]
            RF_CH4_Bio=[]
            RF_N2O_Bio=[]
            RF_CO2_LH2=[]
            RF_CO2_non_fossil_LH2=[]
            RF_CO2_capture_LH2=[]
            RF_CH4_LH2=[]
            RF_N2O_LH2=[]
            PPM_fossil_FK=[]
            PPM_non_fossil_FK=[]
            PPM_capture_FK=[]
            PPM_fossil_PtL=[]
            PPM_non_fossil_PtL=[]
            PPM_capture_PtL=[]
            PPM_fossil_Bio=[]
            PPM_non_fossil_Bio=[]
            PPM_capture_Bio=[]
            PPM_fossil_LH2=[]
            PPM_non_fossil_LH2=[]
            PPM_capture_LH2=[]
            list_of_RF_FK=[]
            list_of_RF_PtL=[]
            list_of_RF_Bio=[]
            list_of_RF_LH2=[]
            list_of_CO2_FK=[] #PPM
            list_of_CO2_PtL=[] #PPM
            list_of_CO2_Bio=[] #PPM
            list_of_CO2_LH2=[] #PPM
            list_of_CO2_emissions_FK=[] #KGCO2
            list_of_CO2_emissions_PtL=[] #KGCO2
            list_of_CO2_emissions_Bio=[] #KGCO2
            list_of_CO2_emissions_LH2=[] #KGCO2
            list_of_BII_CC_fk=[]
            list_of_BII_CC_ptl=[]
            list_of_BII_CC_bio=[]
            list_of_BII_CC_lh2=[]
            i=0
            while i<7:
                fossil_kerosene_CO2_fossil.append(FK[i]*CO2_FK[0])
                fossil_kerosene_CO2_non_fossil.append(FK[i]*CO2_non_fossil_FK[0])
                fossil_kerosene_CO2_capture.append(FK[i]*CO2_capture_FK[0])
                fossil_kerosene_CH4.append(FK[i]*CH4_FK[0])
                fossil_kerosene_N20.append(FK[i]*N2O_FK[0])
                PtL_CO2_fossil.append(PtL[i]*CO2_PtL[0])
                PtL_CO2_non_fossil.append(PtL[i]*CO2_non_fossil_PtL[0])
                PtL_CO2_capture.append(PtL[i]*CO2_capture_PtL[0])
                PtL_CH4.append(PtL[i]*CH4_PtL[0])
                PtL_N20.append(PtL[i]*N2O_PtL[0])
                Bio_CO2_fossil.append(Bio[i]*CO2_Bio[0])
                Bio_CO2_non_fossil.append(Bio[i]*CO2_non_fossil_Bio[0])
                Bio_CO2_capture.append(Bio[i]*CO2_capture_Bio[0])
                Bio_CH4.append(Bio[i]*CH4_Bio[0])
                Bio_N20.append(Bio[i]*N2O_Bio[0])
                LH2_CO2_fossil.append(LH2[i]*CO2_LH2[0])
                LH2_CO2_non_fossil.append(LH2[i]*CO2_non_fossil_LH2[0])
                LH2_CO2_capture.append(LH2[i]*CO2_capture_LH2[0])
                LH2_CH4.append(LH2[i]*CH4_LH2[0])
                LH2_N20.append(LH2[i]*N2O_LH2[0])
                i=i+1
            while i<17:
                fossil_kerosene_CO2_fossil.append(FK[i]*CO2_FK[1])
                fossil_kerosene_CO2_non_fossil.append(FK[i]*CO2_non_fossil_FK[1])
                fossil_kerosene_CO2_capture.append(FK[i]*CO2_capture_FK[1])
                fossil_kerosene_CH4.append(FK[i]*CH4_FK[1])
                fossil_kerosene_N20.append(FK[i]*N2O_FK[1])
                PtL_CO2_fossil.append(PtL[i]*CO2_PtL[1])
                PtL_CO2_non_fossil.append(PtL[i]*CO2_non_fossil_PtL[1])
                PtL_CO2_capture.append(PtL[i]*CO2_capture_PtL[1])
                PtL_CH4.append(PtL[i]*CH4_PtL[1])
                PtL_N20.append(PtL[i]*N2O_PtL[1])
                Bio_CO2_fossil.append(Bio[i]*CO2_Bio[1])
                Bio_CO2_non_fossil.append(Bio[i]*CO2_non_fossil_Bio[1])
                Bio_CO2_capture.append(Bio[i]*CO2_capture_Bio[1])
                Bio_CH4.append(Bio[i]*CH4_Bio[1])
                Bio_N20.append(Bio[i]*N2O_Bio[1])
                LH2_CO2_fossil.append(LH2[i]*CO2_LH2[1])
                LH2_CO2_non_fossil.append(LH2[i]*CO2_non_fossil_LH2[1])
                LH2_CO2_capture.append(LH2[i]*CO2_capture_LH2[1])
                LH2_CH4.append(LH2[i]*CH4_LH2[1])
                LH2_N20.append(LH2[i]*N2O_LH2[1])
                i=i+1
            while i<27:
                fossil_kerosene_CO2_fossil.append(FK[i]*CO2_FK[2])
                fossil_kerosene_CO2_non_fossil.append(FK[i]*CO2_non_fossil_FK[2])
                fossil_kerosene_CO2_capture.append(FK[i]*CO2_capture_FK[2])
                fossil_kerosene_CH4.append(FK[i]*CH4_FK[2])
                fossil_kerosene_N20.append(FK[i]*N2O_FK[2])
                PtL_CO2_fossil.append(PtL[i]*CO2_PtL[2])
                PtL_CO2_non_fossil.append(PtL[i]*CO2_non_fossil_PtL[2])
                PtL_CO2_capture.append(PtL[i]*CO2_capture_PtL[2])
                PtL_CH4.append(PtL[i]*CH4_PtL[2])
                PtL_N20.append(PtL[i]*N2O_PtL[2])
                Bio_CO2_fossil.append(Bio[i]*CO2_Bio[2])
                Bio_CO2_non_fossil.append(Bio[i]*CO2_non_fossil_Bio[2])
                Bio_CO2_capture.append(Bio[i]*CO2_capture_Bio[2])
                Bio_CH4.append(Bio[i]*CH4_Bio[2])
                Bio_N20.append(Bio[i]*N2O_Bio[2])
                LH2_CO2_fossil.append(LH2[i]*CO2_LH2[2])
                LH2_CO2_non_fossil.append(LH2[i]*CO2_non_fossil_LH2[2])
                LH2_CO2_capture.append(LH2[i]*CO2_capture_LH2[2])
                LH2_CH4.append(LH2[i]*CH4_LH2[2])
                LH2_N20.append(LH2[i]*N2O_LH2[2])
                i=i+1
            while i<28:
                fossil_kerosene_CO2_fossil.append(FK[i]*CO2_FK[3])
                fossil_kerosene_CO2_non_fossil.append(FK[i]*CO2_non_fossil_FK[3])
                fossil_kerosene_CO2_capture.append(FK[i]*CO2_capture_FK[3])
                fossil_kerosene_CH4.append(FK[i]*CH4_FK[3])
                fossil_kerosene_N20.append(FK[i]*N2O_FK[3])
                PtL_CO2_fossil.append(PtL[i]*CO2_PtL[3])
                PtL_CO2_non_fossil.append(PtL[i]*CO2_non_fossil_PtL[3])
                PtL_CO2_capture.append(PtL[i]*CO2_capture_PtL[3])
                PtL_CH4.append(PtL[i]*CH4_PtL[3])
                PtL_N20.append(PtL[i]*N2O_PtL[3])
                Bio_CO2_fossil.append(Bio[i]*CO2_Bio[3])
                Bio_CO2_non_fossil.append(Bio[i]*CO2_non_fossil_Bio[3])
                Bio_CO2_capture.append(Bio[i]*CO2_capture_Bio[3])
                Bio_CH4.append(Bio[i]*CH4_Bio[3])
                Bio_N20.append(Bio[i]*N2O_Bio[3])
                LH2_CO2_fossil.append(LH2[i]*CO2_LH2[3])
                LH2_CO2_non_fossil.append(LH2[i]*CO2_non_fossil_LH2[3])
                LH2_CO2_capture.append(LH2[i]*CO2_capture_LH2[3])
                LH2_CH4.append(LH2[i]*CH4_LH2[3])
                LH2_N20.append(LH2[i]*N2O_LH2[3])
                i=i+1
                
            #Emissions totales CO2 year-by-year (kgCO2)
            for i in range(len(fossil_kerosene_CO2_fossil)):
                list_of_CO2_emissions_FK.append(fossil_kerosene_CO2_fossil[i]+fossil_kerosene_CO2_non_fossil[i]+fossil_kerosene_CO2_capture[i])
                list_of_CO2_emissions_PtL.append(PtL_CO2_fossil[i]+PtL_CO2_non_fossil[i]+PtL_CO2_capture[i])
                list_of_CO2_emissions_Bio.append(Bio_CO2_fossil[i]+Bio_CO2_non_fossil[i]+Bio_CO2_capture[i])
                list_of_CO2_emissions_LH2.append(LH2_CO2_fossil[i]+LH2_CO2_non_fossil[i]+LH2_CO2_capture[i])
            
            #Radiative forcing
            RF_CO2_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[0])
            RF_CO2_non_fossil_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[1])
            RF_CO2_capture_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[2])
            RF_CH4_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[3])
            RF_N2O_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[4])
            RF_CO2_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[5])
            RF_CO2_non_fossil_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[6])
            RF_CO2_capture_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[7])
            RF_CH4_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[8])
            RF_N2O_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[9])
            RF_CO2_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[10])
            RF_CO2_non_fossil_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[11])
            RF_CO2_capture_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[12])
            RF_CH4_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[13])
            RF_N2O_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[14])
            RF_CO2_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[15])
            RF_CO2_non_fossil_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[16])
            RF_CO2_capture_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[17])
            RF_CH4_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[18])
            RF_N2O_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[19])
            for i in range(len(fossil_kerosene_CO2_fossil)):
                list_of_RF_FK.append(RF_CO2_FK[i]+RF_CO2_non_fossil_FK[i]+RF_CO2_capture_FK[i]+RF_CH4_FK[i]+RF_N2O_FK[i])
                list_of_RF_PtL.append(RF_CO2_PtL[i]+RF_CO2_non_fossil_PtL[i]+RF_CO2_capture_PtL[i]+RF_CH4_PtL[i]+RF_N2O_PtL[i])
                list_of_RF_Bio.append(RF_CO2_Bio[i]+RF_CO2_non_fossil_Bio[i]+RF_CO2_capture_Bio[i]+RF_CH4_Bio[i]+RF_N2O_Bio[i])
                list_of_RF_LH2.append(RF_CO2_LH2[i]+RF_CO2_non_fossil_LH2[i]+RF_CO2_capture_LH2[i]+RF_CH4_LH2[i]+RF_N2O_LH2[i])
                list_of_BII_CC_fk.append((RF_CO2_FK[i]+RF_CO2_non_fossil_FK[i]+RF_CO2_capture_FK[i]+RF_CH4_FK[i]+RF_N2O_FK[i])*CF_BII_climate)
                list_of_BII_CC_ptl.append((RF_CO2_PtL[i]+RF_CO2_non_fossil_PtL[i]+RF_CO2_capture_PtL[i]+RF_CH4_PtL[i]+RF_N2O_PtL[i])*CF_BII_climate)
                list_of_BII_CC_bio.append((RF_CO2_Bio[i]+RF_CO2_non_fossil_Bio[i]+RF_CO2_capture_Bio[i]+RF_CH4_Bio[i]+RF_N2O_Bio[i])*CF_BII_climate)
                list_of_BII_CC_lh2.append((RF_CO2_LH2[i]+RF_CO2_non_fossil_LH2[i]+RF_CO2_capture_LH2[i]+RF_CH4_LH2[i]+RF_N2O_LH2[i])*CF_BII_climate)
            
            
            #Atmospheric concentration
            PPM_fossil_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[20])
            PPM_non_fossil_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[21])
            PPM_capture_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[22])
            PPM_fossil_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[23])
            PPM_non_fossil_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[24])
            PPM_capture_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[25])
            PPM_fossil_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[26])
            PPM_non_fossil_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[27])
            PPM_capture_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[28])
            PPM_fossil_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[29])
            PPM_non_fossil_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[30])
            PPM_capture_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[31])
            for i in range(len(fossil_kerosene_CO2_fossil)):
                list_of_CO2_FK.append(PPM_fossil_FK[i]+PPM_non_fossil_FK[i]+PPM_capture_FK[i])
                list_of_CO2_PtL.append(PPM_fossil_PtL[i]+PPM_non_fossil_PtL[i]+PPM_capture_PtL[i])
                list_of_CO2_Bio.append(PPM_fossil_Bio[i]+PPM_non_fossil_Bio[i]+PPM_capture_Bio[i])
                list_of_CO2_LH2.append(PPM_fossil_LH2[i]+PPM_non_fossil_LH2[i]+PPM_capture_LH2[i])
            
                        
    
            #Fossil kerosene
            AESA_FK_N=aesa(SoSOS_FHN_N*SOS_N, N_FK_impact)
            AESA_FK_P=aesa(SoSOS_FHN_P*SOS_P, P_FK_impact)
            AESA_FK_O3=aesa(SoSOS_FHN_O3*SOS_O3, O3_FK_impact)
            AESA_FK_BII_DLU=aesa(SoSOS_FHN_BII*SOS_BII, BII_DLU_FK_impact)
            AESA_FK_BII_CC=aesa(SoSOS_FHN_BII*SOS_BII, list_of_BII_CC_fk)
            AESA_FK_FWU=aesa(SoSOS_FHN_FWU*SOS_FWU, FWU_FK_impact)
            AESA_FK_RF=aesa(SoSOS_FHN_CC*SOS_RF,list_of_RF_FK)
            AESA_FK_PPM=aesa(SoSOS_FHN_CC*SOS_PPM,list_of_CO2_FK)
    
            #PtL
            AESA_PtL_N=aesa(SoSOS_FHN_N*SOS_N, N_PtL_impact)
            AESA_PtL_P=aesa(SoSOS_FHN_P*SOS_P, P_PtL_impact)
            AESA_PtL_O3=aesa(SoSOS_FHN_O3*SOS_O3, O3_PtL_impact)
            AESA_PtL_BII_DLU=aesa(SoSOS_FHN_BII*SOS_BII, BII_DLU_PtL_impact)
            AESA_PtL_BII_CC=aesa(SoSOS_FHN_BII*SOS_BII,list_of_BII_CC_ptl)
            AESA_PtL_FWU=aesa(SoSOS_FHN_FWU*SOS_FWU, FWU_PtL_impact)
            AESA_PtL_RF=aesa(SoSOS_FHN_CC*SOS_RF,list_of_RF_PtL)
            AESA_PtL_PPM=aesa(SoSOS_FHN_CC*SOS_PPM,list_of_CO2_PtL)

            #Bio
            AESA_Bio_N=aesa(SoSOS_FHN_N*SOS_N, N_Bio_impact)
            AESA_Bio_P=aesa(SoSOS_FHN_P*SOS_P, P_Bio_impact)
            AESA_Bio_O3=aesa(SoSOS_FHN_O3*SOS_O3, O3_Bio_impact)
            AESA_Bio_BII_DLU=aesa(SoSOS_FHN_BII*SOS_BII, BII_DLU_Bio_impact)
            AESA_Bio_BII_CC=aesa(SoSOS_FHN_BII*SOS_BII, list_of_BII_CC_bio)
            AESA_Bio_FWU=aesa(SoSOS_FHN_FWU*SOS_FWU, FWU_Bio_impact)
            AESA_Bio_RF=aesa(SoSOS_FHN_CC*SOS_RF,list_of_RF_Bio)
            AESA_Bio_PPM=aesa(SoSOS_FHN_CC*SOS_PPM,list_of_CO2_Bio)

            #LH2
            AESA_LH2_N=aesa(SoSOS_FHN_N*SOS_N, N_LH2_impact)
            AESA_LH2_P=aesa(SoSOS_FHN_P*SOS_P, P_LH2_impact)
            AESA_LH2_O3=aesa(SoSOS_FHN_O3*SOS_O3, O3_LH2_impact)
            AESA_LH2_BII_DLU=aesa(SoSOS_FHN_BII*SOS_BII, BII_DLU_LH2_impact)
            AESA_LH2_BII_CC=aesa(SoSOS_FHN_BII*SOS_BII, list_of_BII_CC_lh2)
            AESA_LH2_FWU=aesa(SoSOS_FHN_FWU*SOS_FWU, FWU_LH2_impact)
            AESA_LH2_RF=aesa(SoSOS_FHN_CC*SOS_RF,list_of_RF_LH2)
            AESA_LH2_PPM=aesa(SoSOS_FHN_CC*SOS_PPM,list_of_CO2_LH2)

            scenario.append(nom)
            nature_ptl.append(ptl)
            nature_bio.append(bio)
            n_fk_final.append((AESA_FK_N[27]))
            p_fk_final.append((AESA_FK_P[27]))
            lsc_fk_final.append(LSC_FK_impact_2050/(SoSOS_FHN_LSC*SOS_LSC))
            o3_fk_final.append((AESA_FK_O3[27]))
            bii_dlu_fk_final.append((AESA_FK_BII_DLU[27]))
            fwu_fk_final.append((AESA_FK_FWU[27]))
            n_ptl_final.append((AESA_PtL_N[27]))
            p_ptl_final.append((AESA_PtL_P[27]))
            lsc_ptl_final.append(LSC_PtL_impact_2050/(SoSOS_FHN_LSC*SOS_LSC))
            o3_ptl_final.append((AESA_PtL_O3[27]))
            bii_dlu_ptl_final.append((AESA_PtL_BII_DLU[27]))
            fwu_ptl_final.append((AESA_PtL_FWU[27]))
            n_bio_final.append((AESA_Bio_N[27]))
            p_bio_final.append((AESA_Bio_P[27]))
            lsc_bio_final.append(LSC_Bio_impact_2050/(SoSOS_FHN_LSC*SOS_LSC))
            o3_bio_final.append((AESA_Bio_O3[27]))
            bii_dlu_bio_final.append((AESA_Bio_BII_DLU[27]))
            fwu_bio_final.append((AESA_Bio_FWU[27]))
            n_lh2_final.append((AESA_LH2_N[27]))
            p_lh2_final.append((AESA_LH2_P[27]))
            lsc_lh2_final.append(LSC_LH2_impact_2050/(SoSOS_FHN_LSC*SOS_LSC))
            o3_lh2_final.append((AESA_LH2_O3[27]))
            bii_dlu_lh2_final.append((AESA_LH2_BII_DLU[27]))
            fwu_lh2_final.append((AESA_LH2_FWU[27]))
            lsc_fk_historical.append((historical_fossil_kerosene()*LSC_FK[0]/(SoSOS_FHN_LSC*SOS_LSC)))
            rf_historical.append((historical_rf_2018+historical_rf_2019_2023)/(SoSOS_FHN_CC*SOS_RF))
            rf_fk_final.append((AESA_FK_RF[27]))
            ppm_historical.append((historical_ppm_2018+historical_ppm_2019_2023)/(SoSOS_FHN_CC*SOS_PPM))
            ppm_fk_final.append((AESA_FK_PPM[27]))
            rf_ptl_final.append((AESA_PtL_RF[27]))
            ppm_ptl_final.append((AESA_PtL_PPM[27]))
            rf_bio_final.append((AESA_Bio_RF[27]))
            ppm_bio_final.append((AESA_Bio_PPM[27]))
            rf_lh2_final.append((AESA_LH2_RF[27]))
            ppm_lh2_final.append((AESA_LH2_PPM[27]))
            bii_clim_historical.append(((historical_rf_2018+historical_rf_2019_2023)*CF_BII_climate)/(SoSOS_FHN_BII*SOS_BII))
            bii_clim_fk_final.append((AESA_FK_BII_CC[27]))
            bii_clim_ptl_final.append((AESA_PtL_BII_CC[27]))
            bii_clim_bio_final.append((AESA_Bio_BII_CC[27]))
            bii_clim_lh2_final.append((AESA_LH2_BII_CC[27]))
          
            

In [ ]:
list_of_list=[scenario,
              nature_ptl,
              nature_bio,
              n_fk_final,
              n_ptl_final,
              n_bio_final,
              n_lh2_final,
              p_fk_final,
              p_ptl_final,
              p_bio_final,
              p_lh2_final,
              lsc_fk_historical,
              lsc_fk_final,
              lsc_ptl_final,
              lsc_bio_final,
              lsc_lh2_final,
              o3_fk_final,
              o3_ptl_final,
              o3_bio_final,
              o3_lh2_final,
              bii_dlu_fk_final,
              bii_dlu_ptl_final,
              bii_dlu_bio_final,
              bii_dlu_lh2_final,
              bii_clim_historical,
              bii_clim_fk_final,
              bii_clim_ptl_final,
              bii_clim_bio_final,
              bii_clim_lh2_final,
              fwu_fk_final,                   
              fwu_ptl_final,
              fwu_bio_final,
              fwu_lh2_final,
              rf_historical,
              rf_fk_final,
              rf_ptl_final,
              rf_bio_final,
              rf_lh2_final,
              ppm_historical,
              ppm_fk_final,
              ppm_ptl_final,
              ppm_bio_final,  
              ppm_lh2_final]
df = pd.DataFrame(list_of_list).transpose()
df.columns=['scenario',
                  'nature_ptl',
                  'nature_bio',
                  'n_fk_final',
                  'n_ptl_final',
                  'n_bio_final',
                  'n_lh2_final',
                  'p_fk_final',
                  'p_ptl_final',
                  'p_bio_final',
                  'p_lh2_final',
                  'lsc_fk_historical',
                  'lsc_fk_final',
                  'lsc_ptl_final',
                  'lsc_bio_final',
                  'lsc_lh2_final',
                  'o3_fk_final',
                  'o3_ptl_final',
                  'o3_bio_final',
                  'o3_lh2_final',
                  'bii_dlu_fk_final',
                  'bii_dlu_ptl_final',
                  'bii_dlu_bio_final',
                  'bii_dlu_lh2_final',
                  'bii_clim_historical',
                  'bii_clim_fk_final',
                  'bii_clim_ptl_final',
                  'bii_clim_bio_final',
                  'bii_clim_lh2_final',
                  'fwu_fk_final',                   
                  'fwu_ptl_final',
                  'fwu_bio_final',
                  'fwu_lh2_final',
                  'rf_historical',
                  'rf_fk_final',
                  'rf_ptl_final',
                  'rf_bio_final',
                  'rf_lh2_final',
                  'ppm_historical',
                  'ppm_fk_final',
                  'ppm_ptl_final',
                  'ppm_bio_final',
                  'ppm_lh2_final']

In [ ]:
file_name = 'test_automatisation.xlsx'
df.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

# FIGURE 2

In [ ]:
import pandas as pd

# Supposons que df soit votre DataFrame existant

# Définissez l'ordre des index des lignes dans df2
index_order = [0, 56, 184, 48, 176, 90, 218, 91, 219, 95, 223, 122, 250, 154, 282, 314]

# Créez df2 avec les lignes dans l'ordre spécifié
df2 = df.loc[index_order]

# Réinitialisez les index si nécessaire
df2 = df2.reset_index(drop=True)

# Affichez df2
df2


In [ ]:
# Calculer la somme des colonnes spécifiées
bii_historical=df2['bii_clim_historical']
sum_col_1 = df2['bii_dlu_fk_final'] + df2['bii_clim_fk_final']
sum_col_2 = df2['bii_dlu_ptl_final'] + df2['bii_clim_ptl_final']
sum_col_3 = df2['bii_dlu_bio_final'] + df2['bii_clim_bio_final']
sum_col_4 = df2['bii_dlu_lh2_final'] + df2['bii_clim_lh2_final']

# Insérer les colonnes calculées dans df2 après la colonne 28
df2.insert(29,'bii_tot_historical', bii_historical)
df2.insert(30, 'bii_tot_fk', sum_col_1)
df2.insert(31, 'bii_tot_ptl', sum_col_2)
df2.insert(32, 'bii_tot_bio', sum_col_3)
df2.insert(33, 'bii_tot_lh2', sum_col_4)


In [ ]:
df2.to_excel('data_figure2_automated.xlsx')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Supposons que vous avez un DataFrame df2 de dimensions 16x47
# Remplacez df2 par votre DataFrame réel

# Créez une nouvelle colonne contenant la concaténation des colonnes 1, 2 et 3
df2['x_axis'] = ['100FK', '0FK_PtL', '50FK_PtL','0FK_PtL_WG','50_FK_PtL_WG','0FK_Bio_FT','50FK_Bio_FT','0FK_HEFA_UCO','50FK_HEFA_UCO','0FK_HEFA_RS','50FK_HEFA_RS','0FK_PtL&Bio','50FK_PtL&Bio','0FK_Full','50FK_Full','Refuel_EU type']

# Liste de noms pour les titres des figures
figure_titles = ['Atmospheric CO2 concentration', 'N cycle','Land-system change', 'Stratospheric ozone depletion', 'Energy imbalance at TOA','P cycle','Freswhater use','Biosphere integrity - Functional diversity',]

# Nombre de colonnes par ensemble
cols_per_set = [5, 4, 5, 4, 5, 4, 4, 5]

# Indices des colonnes à utiliser pour le calcul de la somme
sum_columns = [[43, 47], [3, 6],  [11, 15], [16, 19], [38, 42], [7, 10], [34, 37], [29, 33]]

# Facteurs de proportionnalité pour les axes y de droite
right_axis_factors = [SoSOS_FHN_CC/SoSOS_GDP, SoSOS_FHN_N/SoSOS_GDP, SoSOS_FHN_LSC/SoSOS_GDP, SoSOS_FHN_O3/SoSOS_GDP,SoSOS_FHN_CC/SoSOS_GDP, SoSOS_FHN_P/SoSOS_GDP, SoSOS_FHN_FWU/SoSOS_GDP, SoSOS_FHN_BII/SoSOS_GDP]

# Définir les palettes de couleurs pour chaque ensemble de colonnes
colors_4_cols = ['dodgerblue', 'goldenrod', 'forestgreen', 'darkmagenta']
colors_5_cols = ['dimgrey', 'dodgerblue', 'goldenrod', 'forestgreen', 'darkmagenta']

# Créez une figure avec une grille de sous-graphiques pour les figures sélectionnées
fig, axs = plt.subplots(4, 2, figsize=(15, 20))

# Liste des indices dans l'ordre a c e g b d f h
indices = ['a', 'c', 'e', 'g', 'b', 'd', 'f', 'h']
# Parcourir les ensembles de colonnes à tracer
for i, (cols, title, index, factor) in enumerate(zip(sum_columns, figure_titles, indices, right_axis_factors)):
    start_col, end_col = cols
    
    # Sélectionnez les colonnes de l'ensemble actuel
    selected_cols = df2.iloc[:, start_col:end_col + 1]
    
    # Calculez la somme des valeurs pour chaque ligne dans cet ensemble de colonnes
    sums = selected_cols.sum(axis=1)
    
    # Calculez les indices de subplot
    row = i % 4
    col = i // 4
    
    # Choisissez la palette de couleurs en fonction du nombre de colonnes cumulées
    if len(selected_cols.columns) == 4:
        colors = colors_4_cols
    else:
        colors = colors_5_cols
    
    # Tracer le graphique à barres cumulées pour cet ensemble de colonnes
    bottom = np.zeros(len(df2))  # Hauteur de départ pour les barres cumulées
    
    for j, col_name in enumerate(selected_cols.columns):
        axs[row, col].bar(df2['x_axis'], selected_cols[col_name], bottom=bottom, label=col_name, color=colors[j])
        bottom += selected_cols[col_name]  # Met à jour la hauteur de départ pour les barres suivantes
    
    # Ajouter un deuxième axe y à droite proportionnel au premier
    ax_right = axs[row, col].twinx()
    ax_right.set_ylim(0, axs[row, col].get_ylim()[1] * factor)
    
    # Formater les étiquettes de l'axe y en pourcentage pour les deux axes y
    vals_left = axs[row, col].get_yticks()
    axs[row, col].set_yticklabels(['{:.0%}'.format(x) for x in vals_left])
    vals_right = ax_right.get_yticks()
    ax_right.set_yticklabels(['{:.0%}'.format(x) for x in vals_right])
    
    
    # Titre de la figure
    axs[row, col].set_title(title)
    axs[row, col].set_ylabel('Level of SoSOS transgression (FHN)')
    ax_right.set_ylabel('Level of SoSOS transgression (GDP)')
    if row != 3:
        axs[row, col].tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    else:
        axs[row, col].tick_params(axis='x', rotation=90)
    
    # Ajouter un espace entre le haut de la plus haute barre et le cadre de la figure
    axs[row, col].set_ylim(top=axs[row, col].get_ylim()[1] * 1.1)
    
    axs[row, col].text(0.02, 1.05, index, transform=axs[row, col].transAxes, fontweight='bold', fontsize=12, va='top', ha='right')
        
# Tracer les lignes horizontales pour y=100%
    axs[row, col].axhline(y=1, color='lawngreen', linestyle='--', linewidth=1.5)
    ax_right.axhline(y=1, color='red', linestyle='--', linewidth=1.5)


# Ajustement automatique du tracé
plt.tight_layout()
plt.savefig('figure2_auto.pdf')
plt.show()

# FIGURE 3

## Nombre de limite planétaire transgressé

In [16]:
def boucle_transgression(trafic_growth,techno_improv):
    nom_scenario=["100FK", "0FK_PtL", "0FK_Bio", "0FK_PtL&Bio", "0FK_Full", "50FK_PtL", "50FK_Bio", "50FK_PtL&Bio", "50FK_Full", "Refuel_EU type"] #, "0FK_PtL", "0FK_Bio", "0FK_PtL&Bio", "0FK_Full", "50FK_PtL", "50FK_Bio", "50FK_PtL&Bio", "50FK_Full", "Refuel_EU type"
    choix_PtL=["wind","PV", "WG", "Mix"]
    choix_Bio=["FT residues", "FT products", "FT mix", "HEFA UCO mix", "HEFA UCO wind", "HEFA UCO PV", "HEFA UCO SMR", "HEFA Rapeseed WG"]
    scenario=[]
    nature_ptl=[]
    nature_bio=[]
    n_fk_final=[]
    p_fk_final=[]
    lsc_fk_final=[]
    o3_fk_final=[]
    bii_dlu_fk_final=[]
    fwu_fk_final=[]
    n_ptl_final=[]
    p_ptl_final=[]
    lsc_ptl_final=[]
    o3_ptl_final=[]
    bii_dlu_ptl_final=[]
    fwu_ptl_final=[]
    n_bio_final=[]
    p_bio_final=[]
    lsc_bio_final=[]
    o3_bio_final=[]
    bii_dlu_bio_final=[]
    fwu_bio_final=[]
    n_lh2_final=[]
    p_lh2_final=[]
    lsc_lh2_final=[]
    o3_lh2_final=[]
    bii_dlu_lh2_final=[]
    fwu_lh2_final=[]
    lsc_fk_historical=[]
    rf_fk_final=[]
    rf_ptl_final=[]
    rf_bio_final=[]
    rf_lh2_final=[]
    ppm_fk_final=[]
    ppm_ptl_final=[]
    ppm_bio_final=[]
    ppm_lh2_final=[]
    bii_clim_historical=[]
    bii_clim_fk_final=[]
    bii_clim_bio_final=[]
    bii_clim_ptl_final=[]
    bii_clim_lh2_final=[]
    rf_historical=[]
    ppm_historical=[]
    for nom in nom_scenario:
        for ptl in choix_PtL:
            for bio in choix_Bio:

                FK=demande_énergétique(2023,2050,8759364769224,trafic_growth,12400000000000/8759364769224,techno_improv,nom)[0]
                PtL=demande_énergétique(2023,2050,8759364769224,trafic_growth,12400000000000/8759364769224,techno_improv,nom)[1]
                Bio=demande_énergétique(2023,2050,8759364769224,trafic_growth,12400000000000/8759364769224,techno_improv,nom)[2]
                LH2=demande_énergétique(2023,2050,8759364769224,trafic_growth,12400000000000/8759364769224,techno_improv,nom)[3]

                N_PtL=selection_AEC(ptl, bio)[0]
                P_PtL=selection_AEC(ptl, bio)[1]
                LSC_PtL=selection_AEC(ptl, bio)[2]
                O3_PtL=selection_AEC(ptl, bio)[3]
                BII_DLU_PtL=selection_AEC(ptl, bio)[4]
                FWU_PtL=selection_AEC(ptl, bio)[5]
                N_Bio=selection_AEC(ptl, bio)[6]
                P_Bio=selection_AEC(ptl, bio)[7]
                LSC_Bio=selection_AEC(ptl, bio)[8]
                O3_Bio=selection_AEC(ptl, bio)[9]
                BII_DLU_Bio=selection_AEC(ptl, bio)[10]
                FWU_Bio=selection_AEC(ptl, bio)[11]
                CO2_PtL=selection_AEC(ptl, bio)[12]
                CO2_non_fossil_PtL=selection_AEC(ptl, bio)[13]
                CO2_capture_PtL=selection_AEC(ptl, bio)[14]
                CH4_PtL=selection_AEC(ptl, bio)[15]
                N2O_PtL=selection_AEC(ptl,bio)[16]
                CO2_Bio=selection_AEC(ptl, bio)[17]
                CO2_non_fossil_Bio=selection_AEC(ptl, bio)[18]
                CO2_capture_Bio=selection_AEC(ptl, bio)[19]
                CH4_Bio=selection_AEC(ptl, bio)[20]
                N2O_Bio=selection_AEC(ptl,bio)[21]

                #N Cycle
                i=0
                N_FK_impact=[]
                N_PtL_impact=[]
                N_Bio_impact=[]
                N_LH2_impact=[]
                while i<7:
                    N_FK_impact.append(FK[i]*N_FK[0])
                    N_PtL_impact.append(PtL[i]*N_PtL[0])
                    N_Bio_impact.append(Bio[i]*N_Bio[0])
                    N_LH2_impact.append(LH2[i]*N_LH2[0])
                    i=i+1
                while i<17:
                    N_FK_impact.append(FK[i]*N_FK[1])
                    N_PtL_impact.append(PtL[i]*N_PtL[1])
                    N_Bio_impact.append(Bio[i]*N_Bio[1])
                    N_LH2_impact.append(LH2[i]*N_LH2[1])
                    i=i+1
                while i<27:
                    N_FK_impact.append(FK[i]*N_FK[2])
                    N_PtL_impact.append(PtL[i]*N_PtL[2])
                    N_Bio_impact.append(Bio[i]*N_Bio[2])
                    N_LH2_impact.append(LH2[i]*N_LH2[2])
                    i=i+1
                while i<28:
                    N_FK_impact.append(FK[i]*N_FK[3])
                    N_PtL_impact.append(PtL[i]*N_PtL[3])
                    N_Bio_impact.append(Bio[i]*N_Bio[3])
                    N_LH2_impact.append(LH2[i]*N_LH2[3])
                    i=i+1


                #P Cycle
                i=0
                P_FK_impact=[]
                P_PtL_impact=[]
                P_Bio_impact=[]
                P_LH2_impact=[]
                while i<7:
                    P_FK_impact.append(FK[i]*P_FK[0])
                    P_PtL_impact.append(PtL[i]*P_PtL[0])
                    P_Bio_impact.append(Bio[i]*P_Bio[0])
                    P_LH2_impact.append(LH2[i]*P_LH2[0])
                    i=i+1
                while i<17:
                    P_FK_impact.append(FK[i]*P_FK[1])
                    P_PtL_impact.append(PtL[i]*P_PtL[1])
                    P_Bio_impact.append(Bio[i]*P_Bio[1])
                    P_LH2_impact.append(LH2[i]*P_LH2[1])
                    i=i+1
                while i<27:
                    P_FK_impact.append(FK[i]*P_FK[2])
                    P_PtL_impact.append(PtL[i]*P_PtL[2])
                    P_Bio_impact.append(Bio[i]*P_Bio[2])
                    P_LH2_impact.append(LH2[i]*P_LH2[2])
                    i=i+1
                while i<28:
                    P_FK_impact.append(FK[i]*P_FK[3])
                    P_PtL_impact.append(PtL[i]*P_PtL[3])
                    P_Bio_impact.append(Bio[i]*P_Bio[3])
                    P_LH2_impact.append(LH2[i]*P_LH2[3])
                    i=i+1


                #Land system change
                i=0
                LSC_FK_impact=[]
                LSC_PtL_impact=[]
                LSC_Bio_impact=[]
                LSC_LH2_impact=[]
                while i<7:
                    LSC_FK_impact.append(FK[i]*LSC_FK[0])
                    LSC_PtL_impact.append(PtL[i]*LSC_PtL[0])
                    LSC_Bio_impact.append(Bio[i]*LSC_Bio[0])
                    LSC_LH2_impact.append(LH2[i]*LSC_LH2[0])
                    i=i+1
                while i<17:
                    LSC_FK_impact.append(FK[i]*LSC_FK[1])
                    LSC_PtL_impact.append(PtL[i]*LSC_PtL[1])
                    LSC_Bio_impact.append(Bio[i]*LSC_Bio[1])
                    LSC_LH2_impact.append(LH2[i]*LSC_LH2[1])
                    i=i+1
                while i<27:
                    LSC_FK_impact.append(FK[i]*LSC_FK[2])
                    LSC_PtL_impact.append(PtL[i]*LSC_PtL[2])
                    LSC_Bio_impact.append(Bio[i]*LSC_Bio[2])
                    LSC_LH2_impact.append(LH2[i]*LSC_LH2[2])
                    i=i+1
                while i<28:
                    LSC_FK_impact.append(FK[i]*LSC_FK[3])
                    LSC_PtL_impact.append(PtL[i]*LSC_PtL[3])
                    LSC_Bio_impact.append(Bio[i]*LSC_Bio[3])
                    LSC_LH2_impact.append(LH2[i]*LSC_LH2[3])
                    i=i+1
                LSC_FK_impact_2050=sum(LSC_FK_impact)
                LSC_PtL_impact_2050=sum(LSC_PtL_impact)
                LSC_Bio_impact_2050=sum(LSC_Bio_impact)
                LSC_LH2_impact_2050=sum(LSC_LH2_impact)

                #Stratospheric ozone depletion
                i=0
                O3_FK_impact=[]
                O3_PtL_impact=[]
                O3_Bio_impact=[]
                O3_LH2_impact=[]
                while i<7:
                    O3_FK_impact.append(FK[i]*O3_FK[0])
                    O3_PtL_impact.append(PtL[i]*O3_PtL[0])
                    O3_Bio_impact.append(Bio[i]*O3_Bio[0])
                    O3_LH2_impact.append(LH2[i]*O3_LH2[0])
                    i=i+1
                while i<17:
                    O3_FK_impact.append(FK[i]*O3_FK[1])
                    O3_PtL_impact.append(PtL[i]*O3_PtL[1])
                    O3_Bio_impact.append(Bio[i]*O3_Bio[1])
                    O3_LH2_impact.append(LH2[i]*O3_LH2[1])
                    i=i+1
                while i<27:
                    O3_FK_impact.append(FK[i]*O3_FK[2])
                    O3_PtL_impact.append(PtL[i]*O3_PtL[2])
                    O3_Bio_impact.append(Bio[i]*O3_Bio[2])
                    O3_LH2_impact.append(LH2[i]*O3_LH2[2])
                    i=i+1
                while i<28:
                    O3_FK_impact.append(FK[i]*O3_FK[3])
                    O3_PtL_impact.append(PtL[i]*O3_PtL[3])
                    O3_Bio_impact.append(Bio[i]*O3_Bio[3])
                    O3_LH2_impact.append(LH2[i]*O3_LH2[3])
                    i=i+1
                O3_FK_impact_2050=sum(O3_FK_impact)
                O3_PtL_impact_2050=sum(O3_PtL_impact)
                O3_Bio_impact_2050=sum(O3_Bio_impact)
                O3_LH2_impact_2050=sum(O3_LH2_impact)

                #BII Direct land use
                i=0
                BII_DLU_FK_impact=[]
                BII_DLU_PtL_impact=[]
                BII_DLU_Bio_impact=[]
                BII_DLU_LH2_impact=[]
                while i<7:
                    BII_DLU_FK_impact.append(FK[i]*BII_DLU_FK[0])
                    BII_DLU_PtL_impact.append(PtL[i]*BII_DLU_PtL[0])
                    BII_DLU_Bio_impact.append(Bio[i]*BII_DLU_Bio[0])
                    BII_DLU_LH2_impact.append(LH2[i]*BII_DLU_LH2[0])
                    i=i+1
                while i<17:
                    BII_DLU_FK_impact.append(FK[i]*BII_DLU_FK[1])
                    BII_DLU_PtL_impact.append(PtL[i]*BII_DLU_PtL[1])
                    BII_DLU_Bio_impact.append(Bio[i]*BII_DLU_Bio[1])
                    BII_DLU_LH2_impact.append(LH2[i]*BII_DLU_LH2[1])
                    i=i+1
                while i<27:
                    BII_DLU_FK_impact.append(FK[i]*BII_DLU_FK[2])
                    BII_DLU_PtL_impact.append(PtL[i]*BII_DLU_PtL[2])
                    BII_DLU_Bio_impact.append(Bio[i]*BII_DLU_Bio[2])
                    BII_DLU_LH2_impact.append(LH2[i]*BII_DLU_LH2[2])
                    i=i+1
                while i<28:
                    BII_DLU_FK_impact.append(FK[i]*BII_DLU_FK[3])
                    BII_DLU_PtL_impact.append(PtL[i]*BII_DLU_PtL[3])
                    BII_DLU_Bio_impact.append(Bio[i]*BII_DLU_Bio[3])
                    BII_DLU_LH2_impact.append(LH2[i]*BII_DLU_LH2[3])
                    i=i+1


                #Freshwater use
                i=0
                FWU_FK_impact=[]
                FWU_PtL_impact=[]
                FWU_Bio_impact=[]
                FWU_LH2_impact=[]
                while i<7:
                    FWU_FK_impact.append(FK[i]*FWU_FK[0])
                    FWU_PtL_impact.append(PtL[i]*FWU_PtL[0])
                    FWU_Bio_impact.append(Bio[i]*FWU_Bio[0])
                    FWU_LH2_impact.append(LH2[i]*FWU_LH2[0])
                    i=i+1  
                while i<17:
                    FWU_FK_impact.append(FK[i]*FWU_FK[1])
                    FWU_PtL_impact.append(PtL[i]*FWU_PtL[1])
                    FWU_Bio_impact.append(Bio[i]*FWU_Bio[1])
                    FWU_LH2_impact.append(LH2[i]*FWU_LH2[1])
                    i=i+1 
                while i<27:
                    FWU_FK_impact.append(FK[i]*FWU_FK[2])
                    FWU_PtL_impact.append(PtL[i]*FWU_PtL[2])
                    FWU_Bio_impact.append(Bio[i]*FWU_Bio[2])
                    FWU_LH2_impact.append(LH2[i]*FWU_LH2[2])
                    i=i+1 
                while i<28:
                    FWU_FK_impact.append(FK[i]*FWU_FK[3])
                    FWU_PtL_impact.append(PtL[i]*FWU_PtL[3])
                    FWU_Bio_impact.append(Bio[i]*FWU_Bio[3])
                    FWU_LH2_impact.append(LH2[i]*FWU_LH2[3])
                    i=i+1 


                #Climate change
                fossil_kerosene_CO2_fossil=[]
                fossil_kerosene_CO2_non_fossil=[]
                fossil_kerosene_CO2_capture=[]
                fossil_kerosene_CH4=[]
                fossil_kerosene_N20=[]
                PtL_CO2_fossil=[]
                PtL_CO2_non_fossil=[]
                PtL_CO2_capture=[]
                PtL_CH4=[]
                PtL_N20=[]
                Bio_CO2_fossil=[]
                Bio_CO2_non_fossil=[]
                Bio_CO2_capture=[]
                Bio_CH4=[]
                Bio_N20=[]
                LH2_CO2_fossil=[]
                LH2_CO2_non_fossil=[]
                LH2_CO2_capture=[]
                LH2_CH4=[]
                LH2_N20=[]
                RF_CO2_FK=[]
                RF_CO2_non_fossil_FK=[]
                RF_CO2_capture_FK=[]
                RF_CH4_FK=[]
                RF_N2O_FK=[]
                RF_CO2_PtL=[]
                RF_CO2_non_fossil_PtL=[]
                RF_CO2_capture_PtL=[]
                RF_CH4_PtL=[]
                RF_N2O_PtL=[]
                RF_CO2_Bio=[]
                RF_CO2_non_fossil_Bio=[]
                RF_CO2_capture_Bio=[]
                RF_CH4_Bio=[]
                RF_N2O_Bio=[]
                RF_CO2_LH2=[]
                RF_CO2_non_fossil_LH2=[]
                RF_CO2_capture_LH2=[]
                RF_CH4_LH2=[]
                RF_N2O_LH2=[]
                PPM_fossil_FK=[]
                PPM_non_fossil_FK=[]
                PPM_capture_FK=[]
                PPM_fossil_PtL=[]
                PPM_non_fossil_PtL=[]
                PPM_capture_PtL=[]
                PPM_fossil_Bio=[]
                PPM_non_fossil_Bio=[]
                PPM_capture_Bio=[]
                PPM_fossil_LH2=[]
                PPM_non_fossil_LH2=[]
                PPM_capture_LH2=[]
                list_of_RF_FK=[]
                list_of_RF_PtL=[]
                list_of_RF_Bio=[]
                list_of_RF_LH2=[]
                list_of_CO2_FK=[] #PPM
                list_of_CO2_PtL=[] #PPM
                list_of_CO2_Bio=[] #PPM
                list_of_CO2_LH2=[] #PPM
                list_of_CO2_emissions_FK=[] #KGCO2
                list_of_CO2_emissions_PtL=[] #KGCO2
                list_of_CO2_emissions_Bio=[] #KGCO2
                list_of_CO2_emissions_LH2=[] #KGCO2
                list_of_BII_CC_fk=[]
                list_of_BII_CC_ptl=[]
                list_of_BII_CC_bio=[]
                list_of_BII_CC_lh2=[]
                i=0
                while i<7:
                    fossil_kerosene_CO2_fossil.append(FK[i]*CO2_FK[0])
                    fossil_kerosene_CO2_non_fossil.append(FK[i]*CO2_non_fossil_FK[0])
                    fossil_kerosene_CO2_capture.append(FK[i]*CO2_capture_FK[0])
                    fossil_kerosene_CH4.append(FK[i]*CH4_FK[0])
                    fossil_kerosene_N20.append(FK[i]*N2O_FK[0])
                    PtL_CO2_fossil.append(PtL[i]*CO2_PtL[0])
                    PtL_CO2_non_fossil.append(PtL[i]*CO2_non_fossil_PtL[0])
                    PtL_CO2_capture.append(PtL[i]*CO2_capture_PtL[0])
                    PtL_CH4.append(PtL[i]*CH4_PtL[0])
                    PtL_N20.append(PtL[i]*N2O_PtL[0])
                    Bio_CO2_fossil.append(Bio[i]*CO2_Bio[0])
                    Bio_CO2_non_fossil.append(Bio[i]*CO2_non_fossil_Bio[0])
                    Bio_CO2_capture.append(Bio[i]*CO2_capture_Bio[0])
                    Bio_CH4.append(Bio[i]*CH4_Bio[0])
                    Bio_N20.append(Bio[i]*N2O_Bio[0])
                    LH2_CO2_fossil.append(LH2[i]*CO2_LH2[0])
                    LH2_CO2_non_fossil.append(LH2[i]*CO2_non_fossil_LH2[0])
                    LH2_CO2_capture.append(LH2[i]*CO2_capture_LH2[0])
                    LH2_CH4.append(LH2[i]*CH4_LH2[0])
                    LH2_N20.append(LH2[i]*N2O_LH2[0])
                    i=i+1
                while i<17:
                    fossil_kerosene_CO2_fossil.append(FK[i]*CO2_FK[1])
                    fossil_kerosene_CO2_non_fossil.append(FK[i]*CO2_non_fossil_FK[1])
                    fossil_kerosene_CO2_capture.append(FK[i]*CO2_capture_FK[1])
                    fossil_kerosene_CH4.append(FK[i]*CH4_FK[1])
                    fossil_kerosene_N20.append(FK[i]*N2O_FK[1])
                    PtL_CO2_fossil.append(PtL[i]*CO2_PtL[1])
                    PtL_CO2_non_fossil.append(PtL[i]*CO2_non_fossil_PtL[1])
                    PtL_CO2_capture.append(PtL[i]*CO2_capture_PtL[1])
                    PtL_CH4.append(PtL[i]*CH4_PtL[1])
                    PtL_N20.append(PtL[i]*N2O_PtL[1])
                    Bio_CO2_fossil.append(Bio[i]*CO2_Bio[1])
                    Bio_CO2_non_fossil.append(Bio[i]*CO2_non_fossil_Bio[1])
                    Bio_CO2_capture.append(Bio[i]*CO2_capture_Bio[1])
                    Bio_CH4.append(Bio[i]*CH4_Bio[1])
                    Bio_N20.append(Bio[i]*N2O_Bio[1])
                    LH2_CO2_fossil.append(LH2[i]*CO2_LH2[1])
                    LH2_CO2_non_fossil.append(LH2[i]*CO2_non_fossil_LH2[1])
                    LH2_CO2_capture.append(LH2[i]*CO2_capture_LH2[1])
                    LH2_CH4.append(LH2[i]*CH4_LH2[1])
                    LH2_N20.append(LH2[i]*N2O_LH2[1])
                    i=i+1
                while i<27:
                    fossil_kerosene_CO2_fossil.append(FK[i]*CO2_FK[2])
                    fossil_kerosene_CO2_non_fossil.append(FK[i]*CO2_non_fossil_FK[2])
                    fossil_kerosene_CO2_capture.append(FK[i]*CO2_capture_FK[2])
                    fossil_kerosene_CH4.append(FK[i]*CH4_FK[2])
                    fossil_kerosene_N20.append(FK[i]*N2O_FK[2])
                    PtL_CO2_fossil.append(PtL[i]*CO2_PtL[2])
                    PtL_CO2_non_fossil.append(PtL[i]*CO2_non_fossil_PtL[2])
                    PtL_CO2_capture.append(PtL[i]*CO2_capture_PtL[2])
                    PtL_CH4.append(PtL[i]*CH4_PtL[2])
                    PtL_N20.append(PtL[i]*N2O_PtL[2])
                    Bio_CO2_fossil.append(Bio[i]*CO2_Bio[2])
                    Bio_CO2_non_fossil.append(Bio[i]*CO2_non_fossil_Bio[2])
                    Bio_CO2_capture.append(Bio[i]*CO2_capture_Bio[2])
                    Bio_CH4.append(Bio[i]*CH4_Bio[2])
                    Bio_N20.append(Bio[i]*N2O_Bio[2])
                    LH2_CO2_fossil.append(LH2[i]*CO2_LH2[2])
                    LH2_CO2_non_fossil.append(LH2[i]*CO2_non_fossil_LH2[2])
                    LH2_CO2_capture.append(LH2[i]*CO2_capture_LH2[2])
                    LH2_CH4.append(LH2[i]*CH4_LH2[2])
                    LH2_N20.append(LH2[i]*N2O_LH2[2])
                    i=i+1
                while i<28:
                    fossil_kerosene_CO2_fossil.append(FK[i]*CO2_FK[3])
                    fossil_kerosene_CO2_non_fossil.append(FK[i]*CO2_non_fossil_FK[3])
                    fossil_kerosene_CO2_capture.append(FK[i]*CO2_capture_FK[3])
                    fossil_kerosene_CH4.append(FK[i]*CH4_FK[3])
                    fossil_kerosene_N20.append(FK[i]*N2O_FK[3])
                    PtL_CO2_fossil.append(PtL[i]*CO2_PtL[3])
                    PtL_CO2_non_fossil.append(PtL[i]*CO2_non_fossil_PtL[3])
                    PtL_CO2_capture.append(PtL[i]*CO2_capture_PtL[3])
                    PtL_CH4.append(PtL[i]*CH4_PtL[3])
                    PtL_N20.append(PtL[i]*N2O_PtL[3])
                    Bio_CO2_fossil.append(Bio[i]*CO2_Bio[3])
                    Bio_CO2_non_fossil.append(Bio[i]*CO2_non_fossil_Bio[3])
                    Bio_CO2_capture.append(Bio[i]*CO2_capture_Bio[3])
                    Bio_CH4.append(Bio[i]*CH4_Bio[3])
                    Bio_N20.append(Bio[i]*N2O_Bio[3])
                    LH2_CO2_fossil.append(LH2[i]*CO2_LH2[3])
                    LH2_CO2_non_fossil.append(LH2[i]*CO2_non_fossil_LH2[3])
                    LH2_CO2_capture.append(LH2[i]*CO2_capture_LH2[3])
                    LH2_CH4.append(LH2[i]*CH4_LH2[3])
                    LH2_N20.append(LH2[i]*N2O_LH2[3])
                    i=i+1

                #Emissions totales CO2 year-by-year (kgCO2)
                for i in range(len(fossil_kerosene_CO2_fossil)):
                    list_of_CO2_emissions_FK.append(fossil_kerosene_CO2_fossil[i]+fossil_kerosene_CO2_non_fossil[i]+fossil_kerosene_CO2_capture[i])
                    list_of_CO2_emissions_PtL.append(PtL_CO2_fossil[i]+PtL_CO2_non_fossil[i]+PtL_CO2_capture[i])
                    list_of_CO2_emissions_Bio.append(Bio_CO2_fossil[i]+Bio_CO2_non_fossil[i]+Bio_CO2_capture[i])
                    list_of_CO2_emissions_LH2.append(LH2_CO2_fossil[i]+LH2_CO2_non_fossil[i]+LH2_CO2_capture[i])

                #Radiative forcing
                RF_CO2_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[0])
                RF_CO2_non_fossil_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[1])
                RF_CO2_capture_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[2])
                RF_CH4_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[3])
                RF_N2O_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[4])
                RF_CO2_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[5])
                RF_CO2_non_fossil_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[6])
                RF_CO2_capture_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[7])
                RF_CH4_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[8])
                RF_N2O_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[9])
                RF_CO2_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[10])
                RF_CO2_non_fossil_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[11])
                RF_CO2_capture_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[12])
                RF_CH4_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[13])
                RF_N2O_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[14])
                RF_CO2_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[15])
                RF_CO2_non_fossil_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[16])
                RF_CO2_capture_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[17])
                RF_CH4_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[18])
                RF_N2O_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[19])
                for i in range(len(fossil_kerosene_CO2_fossil)):
                    list_of_RF_FK.append(RF_CO2_FK[i]+RF_CO2_non_fossil_FK[i]+RF_CO2_capture_FK[i]+RF_CH4_FK[i]+RF_N2O_FK[i])
                    list_of_RF_PtL.append(RF_CO2_PtL[i]+RF_CO2_non_fossil_PtL[i]+RF_CO2_capture_PtL[i]+RF_CH4_PtL[i]+RF_N2O_PtL[i])
                    list_of_RF_Bio.append(RF_CO2_Bio[i]+RF_CO2_non_fossil_Bio[i]+RF_CO2_capture_Bio[i]+RF_CH4_Bio[i]+RF_N2O_Bio[i])
                    list_of_RF_LH2.append(RF_CO2_LH2[i]+RF_CO2_non_fossil_LH2[i]+RF_CO2_capture_LH2[i]+RF_CH4_LH2[i]+RF_N2O_LH2[i])
                    list_of_BII_CC_fk.append((RF_CO2_FK[i]+RF_CO2_non_fossil_FK[i]+RF_CO2_capture_FK[i]+RF_CH4_FK[i]+RF_N2O_FK[i])*CF_BII_climate)
                    list_of_BII_CC_ptl.append((RF_CO2_PtL[i]+RF_CO2_non_fossil_PtL[i]+RF_CO2_capture_PtL[i]+RF_CH4_PtL[i]+RF_N2O_PtL[i])*CF_BII_climate)
                    list_of_BII_CC_bio.append((RF_CO2_Bio[i]+RF_CO2_non_fossil_Bio[i]+RF_CO2_capture_Bio[i]+RF_CH4_Bio[i]+RF_N2O_Bio[i])*CF_BII_climate)
                    list_of_BII_CC_lh2.append((RF_CO2_LH2[i]+RF_CO2_non_fossil_LH2[i]+RF_CO2_capture_LH2[i]+RF_CH4_LH2[i]+RF_N2O_LH2[i])*CF_BII_climate)


                #Atmospheric concentration
                PPM_fossil_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[20])
                PPM_non_fossil_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[21])
                PPM_capture_FK=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[22])
                PPM_fossil_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[23])
                PPM_non_fossil_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[24])
                PPM_capture_PtL=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[25])
                PPM_fossil_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[26])
                PPM_non_fossil_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[27])
                PPM_capture_Bio=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[28])
                PPM_fossil_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[29])
                PPM_non_fossil_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[30])
                PPM_capture_LH2=(dynamic_climate_change_impact_assessment(fossil_kerosene_CO2_fossil,fossil_kerosene_CO2_non_fossil,fossil_kerosene_CO2_capture,fossil_kerosene_CH4,fossil_kerosene_N20,PtL_CO2_fossil,PtL_CO2_non_fossil,PtL_CO2_capture,PtL_CH4,PtL_N20,Bio_CO2_fossil,Bio_CO2_non_fossil,Bio_CO2_capture,Bio_CH4,Bio_N20,LH2_CO2_fossil,LH2_CO2_non_fossil,LH2_CO2_capture,LH2_CH4,LH2_N20)[31])
                for i in range(len(fossil_kerosene_CO2_fossil)):
                    list_of_CO2_FK.append(PPM_fossil_FK[i]+PPM_non_fossil_FK[i]+PPM_capture_FK[i])
                    list_of_CO2_PtL.append(PPM_fossil_PtL[i]+PPM_non_fossil_PtL[i]+PPM_capture_PtL[i])
                    list_of_CO2_Bio.append(PPM_fossil_Bio[i]+PPM_non_fossil_Bio[i]+PPM_capture_Bio[i])
                    list_of_CO2_LH2.append(PPM_fossil_LH2[i]+PPM_non_fossil_LH2[i]+PPM_capture_LH2[i])



                #Fossil kerosene
                AESA_FK_N=aesa(SoSOS_FHN_N*SOS_N, N_FK_impact)
                AESA_FK_P=aesa(SoSOS_FHN_P*SOS_P, P_FK_impact)
                AESA_FK_O3=aesa(SoSOS_FHN_O3*SOS_O3, O3_FK_impact)
                AESA_FK_BII_DLU=aesa(SoSOS_FHN_BII*SOS_BII, BII_DLU_FK_impact)
                AESA_FK_BII_CC=aesa(SoSOS_FHN_BII*SOS_BII, list_of_BII_CC_fk)
                AESA_FK_FWU=aesa(SoSOS_FHN_FWU*SOS_FWU, FWU_FK_impact)
                AESA_FK_RF=aesa(SoSOS_FHN_CC*SOS_RF,list_of_RF_FK)
                AESA_FK_PPM=aesa(SoSOS_FHN_CC*SOS_PPM,list_of_CO2_FK)

                #PtL
                AESA_PtL_N=aesa(SoSOS_FHN_N*SOS_N, N_PtL_impact)
                AESA_PtL_P=aesa(SoSOS_FHN_P*SOS_P, P_PtL_impact)
                AESA_PtL_O3=aesa(SoSOS_FHN_O3*SOS_O3, O3_PtL_impact)
                AESA_PtL_BII_DLU=aesa(SoSOS_FHN_BII*SOS_BII, BII_DLU_PtL_impact)
                AESA_PtL_BII_CC=aesa(SoSOS_FHN_BII*SOS_BII,list_of_BII_CC_ptl)
                AESA_PtL_FWU=aesa(SoSOS_FHN_FWU*SOS_FWU, FWU_PtL_impact)
                AESA_PtL_RF=aesa(SoSOS_FHN_CC*SOS_RF,list_of_RF_PtL)
                AESA_PtL_PPM=aesa(SoSOS_FHN_CC*SOS_PPM,list_of_CO2_PtL)

                #Bio
                AESA_Bio_N=aesa(SoSOS_FHN_N*SOS_N, N_Bio_impact)
                AESA_Bio_P=aesa(SoSOS_FHN_P*SOS_P, P_Bio_impact)
                AESA_Bio_O3=aesa(SoSOS_FHN_O3*SOS_O3, O3_Bio_impact)
                AESA_Bio_BII_DLU=aesa(SoSOS_FHN_BII*SOS_BII, BII_DLU_Bio_impact)
                AESA_Bio_BII_CC=aesa(SoSOS_FHN_BII*SOS_BII, list_of_BII_CC_bio)
                AESA_Bio_FWU=aesa(SoSOS_FHN_FWU*SOS_FWU, FWU_Bio_impact)
                AESA_Bio_RF=aesa(SoSOS_FHN_CC*SOS_RF,list_of_RF_Bio)
                AESA_Bio_PPM=aesa(SoSOS_FHN_CC*SOS_PPM,list_of_CO2_Bio)

                #LH2
                AESA_LH2_N=aesa(SoSOS_FHN_N*SOS_N, N_LH2_impact)
                AESA_LH2_P=aesa(SoSOS_FHN_P*SOS_P, P_LH2_impact)
                AESA_LH2_O3=aesa(SoSOS_FHN_O3*SOS_O3, O3_LH2_impact)
                AESA_LH2_BII_DLU=aesa(SoSOS_FHN_BII*SOS_BII, BII_DLU_LH2_impact)
                AESA_LH2_BII_CC=aesa(SoSOS_FHN_BII*SOS_BII, list_of_BII_CC_lh2)
                AESA_LH2_FWU=aesa(SoSOS_FHN_FWU*SOS_FWU, FWU_LH2_impact)
                AESA_LH2_RF=aesa(SoSOS_FHN_CC*SOS_RF,list_of_RF_LH2)
                AESA_LH2_PPM=aesa(SoSOS_FHN_CC*SOS_PPM,list_of_CO2_LH2)

                scenario.append(nom)
                nature_ptl.append(ptl)
                nature_bio.append(bio)
                n_fk_final.append((AESA_FK_N[27]))
                p_fk_final.append((AESA_FK_P[27]))
                lsc_fk_final.append(LSC_FK_impact_2050/(SoSOS_FHN_LSC*SOS_LSC))
                o3_fk_final.append((AESA_FK_O3[27]))
                bii_dlu_fk_final.append((AESA_FK_BII_DLU[27]))
                fwu_fk_final.append((AESA_FK_FWU[27]))
                n_ptl_final.append((AESA_PtL_N[27]))
                p_ptl_final.append((AESA_PtL_P[27]))
                lsc_ptl_final.append(LSC_PtL_impact_2050/(SoSOS_FHN_LSC*SOS_LSC))
                o3_ptl_final.append((AESA_PtL_O3[27]))
                bii_dlu_ptl_final.append((AESA_PtL_BII_DLU[27]))
                fwu_ptl_final.append((AESA_PtL_FWU[27]))
                n_bio_final.append((AESA_Bio_N[27]))
                p_bio_final.append((AESA_Bio_P[27]))
                lsc_bio_final.append(LSC_Bio_impact_2050/(SoSOS_FHN_LSC*SOS_LSC))
                o3_bio_final.append((AESA_Bio_O3[27]))
                bii_dlu_bio_final.append((AESA_Bio_BII_DLU[27]))
                fwu_bio_final.append((AESA_Bio_FWU[27]))
                n_lh2_final.append((AESA_LH2_N[27]))
                p_lh2_final.append((AESA_LH2_P[27]))
                lsc_lh2_final.append(LSC_LH2_impact_2050/(SoSOS_FHN_LSC*SOS_LSC))
                o3_lh2_final.append((AESA_LH2_O3[27]))
                bii_dlu_lh2_final.append((AESA_LH2_BII_DLU[27]))
                fwu_lh2_final.append((AESA_LH2_FWU[27]))
                lsc_fk_historical.append((historical_fossil_kerosene()*LSC_FK[0]/(SoSOS_FHN_LSC*SOS_LSC)))
                rf_historical.append((historical_rf_2018+historical_rf_2019_2023)/(SoSOS_FHN_CC*SOS_RF))
                rf_fk_final.append((AESA_FK_RF[27]))
                ppm_historical.append((historical_ppm_2018+historical_ppm_2019_2023)/(SoSOS_FHN_CC*SOS_PPM))
                ppm_fk_final.append((AESA_FK_PPM[27]))
                rf_ptl_final.append((AESA_PtL_RF[27]))
                ppm_ptl_final.append((AESA_PtL_PPM[27]))
                rf_bio_final.append((AESA_Bio_RF[27]))
                ppm_bio_final.append((AESA_Bio_PPM[27]))
                rf_lh2_final.append((AESA_LH2_RF[27]))
                ppm_lh2_final.append((AESA_LH2_PPM[27]))
                bii_clim_historical.append(((historical_rf_2018+historical_rf_2019_2023)*CF_BII_climate)/(SoSOS_FHN_BII*SOS_BII))
                bii_clim_fk_final.append((AESA_FK_BII_CC[27]))
                bii_clim_ptl_final.append((AESA_PtL_BII_CC[27]))
                bii_clim_bio_final.append((AESA_Bio_BII_CC[27]))
                bii_clim_lh2_final.append((AESA_LH2_BII_CC[27]))
    return scenario, nature_ptl,nature_bio,n_fk_final, n_ptl_final,  n_bio_final,  n_lh2_final,p_fk_final, p_ptl_final, p_bio_final, p_lh2_final, lsc_fk_historical,lsc_fk_final,lsc_ptl_final,lsc_bio_final, lsc_lh2_final,o3_fk_final,o3_ptl_final, o3_bio_final, o3_lh2_final,bii_dlu_fk_final,bii_dlu_ptl_final,bii_dlu_bio_final,bii_dlu_lh2_final,bii_clim_historical,bii_clim_fk_final,bii_clim_ptl_final,bii_clim_bio_final,bii_clim_lh2_final,fwu_fk_final,fwu_ptl_final,fwu_bio_final,fwu_lh2_final,rf_historical,rf_fk_final,rf_ptl_final,rf_bio_final,rf_lh2_final,ppm_historical,ppm_fk_final,ppm_ptl_final,ppm_bio_final, ppm_lh2_final

In [17]:
def boucle_df(list_variables):
    list_of_list=[scenario,
              nature_ptl,
              nature_bio,
              n_fk_final,
              n_ptl_final,
              n_bio_final,
              n_lh2_final,
              p_fk_final,
              p_ptl_final,
              p_bio_final,
              p_lh2_final,
              lsc_fk_historical,
              lsc_fk_final,
              lsc_ptl_final,
              lsc_bio_final,
              lsc_lh2_final,
              o3_fk_final,
              o3_ptl_final,
              o3_bio_final,
              o3_lh2_final,
              bii_dlu_fk_final,
              bii_dlu_ptl_final,
              bii_dlu_bio_final,
              bii_dlu_lh2_final,
              bii_clim_historical,
              bii_clim_fk_final,
              bii_clim_ptl_final,
              bii_clim_bio_final,
              bii_clim_lh2_final,
              fwu_fk_final,                   
              fwu_ptl_final,
              fwu_bio_final,
              fwu_lh2_final,
              rf_historical,
              rf_fk_final,
              rf_ptl_final,
              rf_bio_final,
              rf_lh2_final,
              ppm_historical,
              ppm_fk_final,
              ppm_ptl_final,
              ppm_bio_final,  
              ppm_lh2_final]
    for i in range(len(list_variables)):
        list_of_list[i]=list_variables[i]
    df = pd.DataFrame(list_of_list).transpose()
    df.columns=['scenario',
                      'nature_ptl',
                      'nature_bio',
                      'n_fk_final',
                      'n_ptl_final',
                      'n_bio_final',
                      'n_lh2_final',
                      'p_fk_final',
                      'p_ptl_final',
                      'p_bio_final',
                      'p_lh2_final',
                      'lsc_fk_historical',
                      'lsc_fk_final',
                      'lsc_ptl_final',
                      'lsc_bio_final',
                      'lsc_lh2_final',
                      'o3_fk_final',
                      'o3_ptl_final',
                      'o3_bio_final',
                      'o3_lh2_final',
                      'bii_dlu_fk_final',
                      'bii_dlu_ptl_final',
                      'bii_dlu_bio_final',
                      'bii_dlu_lh2_final',
                      'bii_clim_historical',
                      'bii_clim_fk_final',
                      'bii_clim_ptl_final',
                      'bii_clim_bio_final',
                      'bii_clim_lh2_final',
                      'fwu_fk_final',                   
                      'fwu_ptl_final',
                      'fwu_bio_final',
                      'fwu_lh2_final',
                      'rf_historical',
                      'rf_fk_final',
                      'rf_ptl_final',
                      'rf_bio_final',
                      'rf_lh2_final',
                      'ppm_historical',
                      'ppm_fk_final',
                      'ppm_ptl_final',
                      'ppm_bio_final',
                      'ppm_lh2_final']

    # Définissez l'ordre des index des lignes dans df2
    index_order = [0, 56, 184, 48, 176, 90, 218, 91, 219, 95, 223, 122, 250, 154, 282, 314]

    # Créez df2 avec les lignes dans l'ordre spécifié
    df2 = df.loc[index_order]

    # Réinitialisez les index si nécessaire
    df2 = df2.reset_index(drop=True)
    # Calculer la somme des colonnes spécifiées
    bii_historical=df2['bii_clim_historical']
    sum_col_1 = df2['bii_dlu_fk_final'] + df2['bii_clim_fk_final']
    sum_col_2 = df2['bii_dlu_ptl_final'] + df2['bii_clim_ptl_final']
    sum_col_3 = df2['bii_dlu_bio_final'] + df2['bii_clim_bio_final']
    sum_col_4 = df2['bii_dlu_lh2_final'] + df2['bii_clim_lh2_final']

    # Insérer les colonnes calculées dans df2 après la colonne 28
    df2.insert(29,'bii_tot_historical', bii_historical)
    df2.insert(30, 'bii_tot_fk', sum_col_1)
    df2.insert(31, 'bii_tot_ptl', sum_col_2)
    df2.insert(32, 'bii_tot_bio', sum_col_3)
    df2.insert(33, 'bii_tot_lh2', sum_col_4)

In [18]:
def transgression_PB(df2):
    # Initialiser un tableau pour stocker les transgressions PB pour chaque ligne
    transgressions_PB = []

    # Parcourir chaque ligne du DataFrame
    for index, row in df2.iterrows():
        # Initialiser le nombre de transgressions PB pour cette ligne
        transgression_PB = 0

        # Calculer la somme pour chaque plage de colonnes
        N_FHN = row[3:6].sum()
        P_FHN = row[7:10].sum()
        LSC_FHN = row[11:15].sum()
        O3_FHN = row[16:19].sum()
        BII_FHN = row[29:33].sum()
        RF_FHN = row[38:42].sum()
        PPM_FHN = row[43:47].sum()
        # Vérifier si la somme dépasse 1 et incrémenter transgression_PB si nécessaire
        if N_FHN > 1:
            transgression_PB += 1
        if P_FHN > 1:
            transgression_PB += 1
        if LSC_FHN > 1:
            transgression_PB += 1
        if O3_FHN > 1:
            transgression_PB += 1
        if BII_FHN > 1:
            transgression_PB += 1
        if RF_FHN > 1:
            transgression_PB += 1
        if PPM_FHN > 1:
            transgression_PB += 1

        # Ajouter le nombre de transgressions PB à la liste
        transgressions_PB.append(transgression_PB)
    return transgressions_PB

In [ ]:
traffic_growth = np.linspace(-0.1, 0.1, 9)
list_transgressions_PB = pd.DataFrame()

# Créez une liste temporaire pour stocker les résultats de boucle_transgression
temp_results = [boucle_transgression(trafic, 0.015) for trafic in traffic_growth]

# Utilisez une liste en compréhension pour calculer les transgressions PB
list_transgressions_PB = pd.DataFrame({trafic: transgression_PB(boucle_df(vars)) for trafic, vars in zip(traffic_growth, temp_results)})

# Affichage de votre DataFrame
print(list_transgressions_PB)


In [ ]:
list_transgressions_PB.to_excel('data_figure3_automated.xlsx')

In [ ]:
# Tracer la figure
plt.figure(figsize=(10, 6))

# Tracer une ligne pour chaque ligne du DataFrame
for index, row in list_transgressions_PB.iterrows():
    plt.step(list_transgressions_PB.columns, row, label=df2['scenario'][index])

# Titres et légendes
plt.xlabel("Trafic evolution")
plt.ylabel("# PB transgressed")
plt.legend()

# Rotation des étiquettes de l'axe x pour une meilleure lisibilité
plt.xticks(rotation=90)

# Afficher la figure
plt.tight_layout()
plt.show()